In [ ]:
"""
LINCS 2020数据加载和预处理脚本 - 优化版
修正DOS过滤和时间点匹配问题
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from numba import jit, prange
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings
import gc
from tqdm import tqdm
warnings.filterwarnings('ignore')

def assign_standard_dose(dose_values, method='standard_doses', tolerance_pct=0.1, round_precision=0.5):
    """
    将精确剂量值映射到标准剂量
    
    Parameters:
    -----------
    dose_values : array-like
        原始剂量值
    method : str
        'standard_doses' - 映射到预定义的标准剂量 (0.01, 0.03, 0.1, 0.3, 1, 3, 10 等)
        'rounding' - 四舍五入到指定精度
        'hybrid' - 混合方法：先尝试标准剂量，失败则四舍五入
    tolerance_pct : float
        标准剂量映射的容差百分比 (默认 10%)
    round_precision : float
        四舍五入精度 (默认 0.5 µM)
    
    Returns:
    --------
    np.array : 标准化后的剂量值
    """
    dose_values = np.array(dose_values, dtype=np.float64)
    
    # 定义药理学常用的标准剂量（半对数级别）
    # 这些是实验中最常用的剂量点
    # standard_doses = np.array([
    #    0.001, 0.003, 0.01, 0.03, 0.04, 0.05, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.37, 0.4, 0.5, 
    #    0.75, 0.77, 0.8, 1.0, 1.1, 1.11, 1.2, 1.5, 2.0, 2.3, 2.5, 3.0, 3.3, 3.33, 4.0, 5.0, 
    #    6.0, 7.0, 7.5, 8.0, 10.0, 12.0, 15.0, 20.0, 25.0, 30.0, 50.0, 100.0
    #])
    standard_doses = np.array([1.0, 2.5, 5.0, 10, 15, 20])
        
    
    standardized = np.full_like(dose_values, np.nan)
    
    for i, dose in enumerate(dose_values):
        if np.isnan(dose):
            continue
            
        if method == 'rounding':
            # 方法1：简单四舍五入
            standardized[i] = round(dose / round_precision) * round_precision
            
        elif method == 'standard_doses':
            # 方法2：映射到最近的标准剂量
            # 计算与每个标准剂量的相对差异
            relative_diff = np.abs(standard_doses - dose) / np.maximum(dose, 0.001)
            min_idx = np.argmin(relative_diff)
            
            if relative_diff[min_idx] <= tolerance_pct:
                standardized[i] = standard_doses[min_idx]
            else:
                # 如果差异太大，使用四舍五入
                standardized[i] = round(dose / round_precision) * round_precision
                
        elif method == 'hybrid':
            # 方法3：混合方法
            # 首先尝试映射到标准剂量
            relative_diff = np.abs(standard_doses - dose) / np.maximum(dose, 0.001)
            min_idx = np.argmin(relative_diff)
            
            if relative_diff[min_idx] <= tolerance_pct:
                standardized[i] = standard_doses[min_idx]
            else:
                # 对于非标准剂量，使用智能四舍五入
                # 小剂量用更细的精度，大剂量用更粗的精度
                if dose < 0.1:
                    precision = 0.01
                elif dose < 1:
                    precision = 0.1
                elif dose < 10:
                    precision = 0.5
                else:
                    precision = 1.0
                standardized[i] = round(dose / precision) * precision
    
    return standardized

def assign_standard_dose(dose_values, method='standard_doses', tolerance_pct=0.1, round_precision=0.5):
    """
    将1-20 µM范围内的剂量直接映射到最近的标准剂量
    
    Parameters:
    -----------
    dose_values : array-like
        原始剂量值（单位：µM）
    
    Returns:
    --------
    np.array : 标准化后的剂量值
    
    映射规则:
    - 1-20 µM范围内：映射到最近的标准剂量点
    - <1 µM 或 >20 µM：映射到边界值 (1.0 或 20.0)
    - NaN：保持为NaN
    """
    import numpy as np
    
    dose_values = np.array(dose_values, dtype=np.float64)
    
    # 定义标准剂量点（1-20 µM范围，7个点）
    standard_doses = np.array([10.0])
    
    standardized = np.full_like(dose_values, np.nan)
    
    for i, dose in enumerate(dose_values):
        if np.isnan(dose):
            continue
        
        # 将剂量限制在1-20范围内
        dose_clipped = np.clip(dose, 1, 20)
        
        # 找到最近的标准剂量（使用绝对差值）
        distances = np.abs(standard_doses - dose_clipped)
        min_idx = np.argmin(distances)
        
        standardized[i] = standard_doses[min_idx]
    
    return standardized

class LINCS2020DataLoader:
    """
    加载和预处理LINCS 2020数据 - 优化版
    修正DOS过滤和时间点匹配
    """
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.compound_info = None
        self.inst_info = None  
        self.signatures = None
        self.decompressed_files = []
        
        print(f"\n{'='*80}")
        print(f"🔬 LINCS 2020 Data Loader - Optimized Version")
        print(f"{'='*80}")
        print(f"Data directory: {self.data_dir}")
        print(f"Dataset: Expanded CMap LINCS Resource 2020")
        print(f"{'='*80}\n")
    
    def load_gene_info(self):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / "geneinfo_beta.txt"
        
        if not gene_file.exists():
            raise FileNotFoundError(
                f"❌ Gene info file not found: {gene_file}\n"
                f"   Please download 'geneinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"📖 Loading gene information...")
        print(f"   File: {gene_file.name}")
        
        gene_info = pd.read_csv(gene_file, sep='\t')
        
        print(f"   ✓ Loaded {len(gene_info):,} genes")
        print(f"   ✓ Columns: {list(gene_info.columns)}")
        
        if 'feature_space' in gene_info.columns:
            landmark_mask = gene_info['feature_space'] == 'landmark'
            landmark_genes = gene_info[landmark_mask].copy()
            print(f"   ✓ Using 'feature_space' column to identify landmarks")
        else:
            raise ValueError(
                f"Cannot identify landmark genes. 'feature_space' column not found.\n"
                f"Available columns: {list(gene_info.columns)}"
            )

        print(f"   ✓ Landmark genes: {len(landmark_genes):,}")
        print(f"   ✓ Expected: 978")

        self.landmark_col_indices = np.where(landmark_mask.values)[0]
        print(f"\n   Landmark column indices (first 10): {self.landmark_col_indices[:10]}")

        self.landmark_gene_ids = set(landmark_genes['gene_id'].astype(str).values)
        self.landmark_gene_symbols = set(landmark_genes['gene_symbol'].astype(str).values)

        print(f"   Sample IDs: {list(self.landmark_gene_ids)[:5]}")
        print(f"   Sample symbols: {list(self.landmark_gene_symbols)[:5]}")

        self.gene_info = gene_info
        return gene_info
    
    def load_cell_info(self):
        """加载细胞系信息"""
        cell_file = self.data_dir / "cellinfo_beta.txt"
        
        if not cell_file.exists():
            raise FileNotFoundError(
                f"❌ Cell info file not found: {cell_file}\n"
                f"   Please download 'cellinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"\n📖 Loading cell information...")
        print(f"   File: {cell_file.name}")
        
        cell_info = pd.read_csv(cell_file, sep='\t')
        
        print(f"   ✓ Loaded {len(cell_info):,} cell lines")
        print(f"   ✓ Columns: {list(cell_info.columns[:10])}...")
        
        if 'cell_iname' in cell_info.columns:
            print(f"   ✓ Unique cell lines (cell_iname): {cell_info['cell_iname'].nunique()}")
        
        print(f"\n   Sample cell lines:")
        for cell in cell_info['cell_iname'].head(5).values:
            print(f"     - {cell}")
        
        self.cell_info = cell_info
        return cell_info
    
    def load_compound_info(self):
        """加载化合物信息"""
        compound_file = self.data_dir / "compoundinfo_beta.txt"
        
        if not compound_file.exists():
            print(f"⚠️  Compound info file not found: {compound_file}")
            print(f"   This file is optional but recommended.")
            return None
        
        print(f"\n📖 Loading compound information...")
        print(f"   File: {compound_file.name}")
        
        compound_info = pd.read_csv(compound_file, sep='\t')
        
        print(f"   ✓ Loaded {len(compound_info):,} compounds")
        print(f"   ✓ Columns: {list(compound_info.columns)}")
        
        if 'pert_id' in compound_info.columns:
            print(f"   ✓ Unique perturbagens: {compound_info['pert_id'].nunique()}")
        
        print(f"\n   Sample compounds:")
        for _, row in compound_info.head(3).iterrows():
            name = row.get('cmap_name', 'Unknown')
            print(f"     - {row['pert_id']}: {name}")
        
        self.compound_info = compound_info
        return compound_info
    
    def load_instance_info(self):
        """加载实例信息"""
        inst_file = self.data_dir / "instinfo_beta.txt"

        if not inst_file.exists():
            raise FileNotFoundError(
                f"❌ Instance info file not found: {inst_file}\n"
                f"   Please download 'instinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )

        print(f"\n📖 Loading instance information...")
        print(f"   File: {inst_file.name}")

        inst_info = pd.read_csv(inst_file, sep='\t')

        print(f"   ✓ Loaded {len(inst_info):,} instances")
        print(f"   ✓ Columns: {list(inst_info.columns[:10])}...")

        if 'inst_id' in inst_info.columns:
            self.instance_join_col = 'inst_id'
        elif 'sample_id' in inst_info.columns:
            self.instance_join_col = 'sample_id'
        else:
            raise ValueError(
                f"Neither 'inst_id' nor 'sample_id' found in instinfo_beta.txt. "
                f"Available columns: {list(inst_info.columns)}"
            )

        print(f"   ✓ Using '{self.instance_join_col}' as join key")

        self.inst_info = inst_info
        return inst_info

    def decompress_gctx_file(self, gctx_file):
        """如果GCTX文件被压缩，则解压"""
        gctx_file = Path(gctx_file)
        
        if not gctx_file.exists():
            raise FileNotFoundError(f"GCTX file not found: {gctx_file}")
        
        if str(gctx_file).endswith('.gz'):
            print(f"⚠️  Detected compressed GCTX file: {gctx_file.name}")
            
            decompressed_dir = self.data_dir / "_decompressed"
            decompressed_dir.mkdir(exist_ok=True)
            
            output_path = decompressed_dir / gctx_file.stem
            
            if output_path.exists():
                print(f"✓ Found existing decompressed file: {output_path.name}")
                return str(output_path)
            
            print(f"📦 Decompressing...")
            with gzip.open(gctx_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            print(f"✓ Decompressed to: {output_path}")
            return str(output_path)
        
        return str(gctx_file)
    
    def read_gctx(self, gctx_file, use_landmark_only=True):
        """读取GCTX文件 - 内存优化版"""
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        file_size_gb = Path(gctx_file).stat().st_size / (1024**3)
        print(f"   File size: {file_size_gb:.2f} GB")
        
        if file_size_gb > 50:
            print(f"   ⚠️  Large file detected. Using memory-optimized loading...")
        
        gctx_file = self.decompress_gctx_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            print(f"📊 Inspecting HDF5 structure...")
            print(f"   Available keys: {list(f.keys())}")
            
            if '/0/DATA/0/matrix' in f:
                matrix_dataset = f['/0/DATA/0/matrix']
                row_path = '/0/META/ROW'
                col_path = '/0/META/COL'
            elif '/matrix' in f:
                matrix_dataset = f['/matrix']
                row_path = '/row'
                col_path = '/col'
            else:
                raise ValueError(f"Cannot find matrix in GCTX file. Available keys: {list(f.keys())}")
            
            matrix_shape = matrix_dataset.shape
            print(f"   ✓ Matrix shape: {matrix_shape} (samples × genes)")
            
            print(f"\n📋 Loading metadata...")
            
            # 读取样本元数据
            sample_meta = {}
            if row_path in f:
                print(f"   Reading sample metadata from: {row_path}")
                print(f"   Available row fields: {list(f[row_path].keys())}")
                
                for key in f[row_path].keys():
                    data = f[f'{row_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            sample_meta[key] = data.astype(str)
                    else:
                        sample_meta[key] = data
                
                print(f"   ✓ Loaded {len(sample_meta)} sample metadata fields")
            else:
                raise ValueError(f"Cannot find row metadata at: {row_path}")
            
            # 读取基因元数据
            gene_meta = {}
            if col_path in f:
                print(f"   Reading gene metadata from: {col_path}")
                print(f"   Available col fields: {list(f[col_path].keys())}")
                
                for key in f[col_path].keys():
                    data = f[f'{col_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            gene_meta[key] = data.astype(str)
                    else:
                        gene_meta[key] = data
                
                print(f"   ✓ Loaded {len(gene_meta)} gene metadata fields")
            else:
                raise ValueError(f"Cannot find col metadata at: {col_path}")
            
            sample_df = pd.DataFrame(sample_meta)
            gene_df = pd.DataFrame(gene_meta)

            # 自动检测并交换ROW/COL
            if self.gene_info is not None:
                n_features_expected = len(self.gene_info)
                if len(sample_df) == n_features_expected and len(gene_df) != n_features_expected:
                    print(f"\n   ⚠️ Detected ROW/COL swap. Correcting...")
                    sample_df, gene_df = gene_df, sample_df

            print(f"\n   Sample metadata columns: {list(sample_df.columns[:10])}...")
            print(f"   Gene metadata columns: {list(gene_df.columns)}")
            
            # 确定landmark基因的列索引
            landmark_col_indices = None
            if use_landmark_only:
                print(f"\n🔬 Filtering to landmark genes...")

                if (self.gene_info is None) or (not hasattr(self, "landmark_col_indices")):
                    print(f"   Loading gene info to get landmark indices...")
                    self.load_gene_info()

                if len(self.gene_info) != matrix_shape[1]:
                    print(f"   ⚠️  Warning: geneinfo rows ({len(self.gene_info)}) "
                        f"!= GCTX feature count ({matrix_shape[1]})")
                
                landmark_col_indices = np.array(self.landmark_col_indices, dtype=int)
                print(f"   ✓ Using {len(landmark_col_indices)} landmark features "
                    f"out of {matrix_shape[1]} total")
            
            # 内存优化：分块读取
            print(f"\n🎯 Loading data (memory-optimized)...")
            if landmark_col_indices is not None:
                print(f"   Reading {len(landmark_col_indices)} columns out of {matrix_shape[1]}...")
                chunk_size = 610000
                chunks = []
                
                for start_idx in range(0, matrix_shape[0], chunk_size):
                    end_idx = min(start_idx + chunk_size, matrix_shape[0])
                    print(f"   Loading rows {start_idx:,} to {end_idx:,}... ({end_idx/matrix_shape[0]*100:.1f}%)", end='\r')
                    
                    chunk = matrix_dataset[start_idx:end_idx, landmark_col_indices].astype(np.float32)
                    chunks.append(chunk)
                    
                    if len(chunks) >= 10:
                        print(f"\n   Consolidating chunks...")
                        merged = np.vstack(chunks)
                        chunks = [merged]
                        gc.collect()
                
                print(f"\n   Finalizing matrix...")
                matrix = np.vstack(chunks) if len(chunks) > 1 else chunks[0]
                del chunks
                gc.collect()
            else:
                print(f"   Reading full matrix...")
                matrix = matrix_dataset[:].astype(np.float32)
            
            print(f"   ✓ Final matrix shape: {matrix.shape}")
            print(f"   ✓ Memory usage: {matrix.nbytes / (1024**3):.2f} GB")
            print(f"   ✓ Data type: {matrix.dtype}")
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self):
        """加载Level 4数据"""
        level4_file = self.data_dir / "level4_beta_trt_cp_n1805898x12328.gctx"
        
        if not level4_file.exists():
            pattern = self.data_dir / "level4_beta_trt_cp*.gctx"
            files = glob.glob(str(pattern))
            
            if not files:
                raise FileNotFoundError(
                    f"❌ Level 4 file not found: {level4_file}\n"
                    f"   Please download from: https://clue.io/data/CMap2020#LINCS2020"
                )
            
            level4_file = files[0]
        
        print(f"\n{'='*80}")
        print(f"📖 Loading Level 4 Signatures")
        print(f"{'='*80}")
        print(f"File: {level4_file.name}")
        
        matrix, sample_meta, gene_meta = self.read_gctx(
            level4_file, 
            use_landmark_only=True
        )

        # 用instinfo补齐metadata
        if 'id' in sample_meta.columns:
            sample_meta = sample_meta.rename(columns={'id': 'sample_id'})
        elif 'sample_id' not in sample_meta.columns:
            raise ValueError("Cannot find 'id' or 'sample_id' in GCTX ROW metadata")

        if self.inst_info is None:
            self.load_instance_info()

        inst_info = self.inst_info
        join_col = getattr(self, "instance_join_col", None)
        
        if join_col is None:
            join_col = 'sample_id' if 'sample_id' in inst_info.columns else 'inst_id'

        if join_col not in sample_meta.columns:
            if join_col == 'inst_id' and 'sample_id' in sample_meta.columns:
                print("   ⚠️ Using 'sample_id' instead of 'inst_id'")
                join_col = 'sample_id'
            else:
                raise ValueError(
                    f"Join column '{join_col}' not found in GCTX metadata. "
                    f"Available: {list(sample_meta.columns)}"
                )

        print(f"   🔗 Merging GCTX metadata with instinfo on '{join_col}'...")
        merged_meta = sample_meta.merge(inst_info, on=join_col, how='left')

        if 'pert_id' not in merged_meta.columns:
            raise ValueError("After merging, 'pert_id' is still missing")

        n_missing = merged_meta['pert_id'].isna().sum()
        if n_missing > 0:
            print(f"   ⚠️ {n_missing:,} rows have missing pert_id")

        self.signatures = {
            'matrix': matrix,
            'row_meta': merged_meta,
            'col_meta': gene_meta
        }

        return matrix, merged_meta, gene_meta
    
    def calculate_cosine_similarity_to_nearest_replicate(
        self, 
        matrix: np.ndarray, 
        pert_ids: pd.Series
    ) -> np.ndarray:
        """
        使用Numba JIT编译的超高效版本
        
        计算每个样本与其最近同化合物复制品的余弦相似度
        """

        print(f"\n📊 Calculating similarities (Numba-accelerated)...")
        
        # 预归一化
        matrix_norm = normalize(matrix, norm='l2', axis=1).astype(np.float32)
        
        @jit(nopython=True, parallel=True, fastmath=True)
        def compute_max_similarities(data, indices, n_total):
            """Numba加速的核心计算"""
            n = len(indices)
            result = np.full(n_total, -np.inf, dtype=np.float32)
            
            for i in prange(n):
                idx_i = indices[i]
                vec_i = data[i]
                max_sim = -np.inf
                
                for j in range(n):
                    if i != j:
                        # 点积（已归一化 = 余弦相似度）
                        sim = np.dot(vec_i, data[j])
                        if sim > max_sim:
                            max_sim = sim
                
                result[idx_i] = max_sim
            
            return result
        
        nearest_similarities = np.zeros(len(pert_ids), dtype=np.float32)
        
        # 按化合物分组
        pert_ids_array = pert_ids.values
        unique_perts = np.unique(pert_ids_array)
        
        from tqdm import tqdm
        for pert_id in tqdm(unique_perts, desc="   Computing"):
            mask = pert_ids_array == pert_id
            indices = np.where(mask)[0]
            
            if len(indices) < 2:
                nearest_similarities[indices] = 0.0
                continue
            
            pert_data = matrix_norm[mask]
            sims = compute_max_similarities(pert_data, indices, len(pert_ids))
            nearest_similarities[indices] = sims[indices]
        
        print(f"   ✓ Mean similarity: {nearest_similarities.mean():.4f}")
        return nearest_similarities
    
    def prepare_training_data(
        self, 
        min_observations_per_compound=5,
        min_replicate_similarity=0.12,
        dose_range=(0.1, 20.0),
        valid_timepoints=[6, 24],  # 修改为数值列表
        min_cell_lines=5,
        max_cell_lines=40,
        min_compounds_per_cell=200,
        remove_dos=True
    ):
        """
        准备训练数据
        修正DOS过滤和时间点匹配
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta'].copy()
        col_meta = self.signatures['col_meta']
        
        print(f"\n[DEBUG] row_meta shape: {row_meta.shape}")
        print(f"[DEBUG] columns (first 15): {list(row_meta.columns[:15])}")
        print(f"[DEBUG] Example row:")
        print(row_meta.iloc[0][['sample_id', 'pert_id', 'pert_type', 'cell_iname', 'pert_time', 'pert_dose']])

        print("\n" + "=" * 80)
        print("🔍 DRUGREFLECTOR QUALITY CONTROL PIPELINE - Optimized")
        print("=" * 80)
        print(f"Initial samples: {len(row_meta):,}")
        print(f"Initial memory: {matrix.nbytes / (1024**3):.2f} GB")
        
        print(f"\n📋 Available metadata columns:")
        for col in row_meta.columns[:15]:
            sample_val = row_meta[col].iloc[0] if len(row_meta) > 0 else 'N/A'
            print(f"   - {col}: {sample_val}")
        if len(row_meta.columns) > 15:
            print(f"   ... and {len(row_meta.columns) - 15} more")
        
        # 检查必需字段
        required_fields = ['pert_id']
        missing_fields = [f for f in required_fields if f not in row_meta.columns]
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        # 确定细胞系ID字段
        cell_id_col = None
        for possible_col in ['cell_id', 'cell_iname', 'cell_mfc_name']:
            if possible_col in row_meta.columns:
                cell_id_col = possible_col
                print(f"\n✓ Using '{cell_id_col}' as cell line identifier")
                break
        
        if cell_id_col is None:
            print(f"\n⚠️  Warning: Cannot find cell line identifier")
        
        valid_mask = np.ones(len(row_meta), dtype=bool)
        
        initial_compounds = row_meta['pert_id'].nunique()
        print(f"\nInitial compounds: {initial_compounds:,}")
        
        # ========== Filter 1: Remove DOS compounds (优化) ==========
        if remove_dos:
            print(f"\n{'='*80}")
            print(f"FILTER 1: Remove DOS compounds (keep trt_cp only)")
            print(f"{'='*80}")
            
            # 使用pert_type字段，只保留trt_cp
            if 'pert_type' in row_meta.columns:
                # 检查pert_type的值分布
                print(f"   pert_type value counts:")
                pert_type_counts = row_meta['pert_type'].value_counts()
                for ptype, count in pert_type_counts.items():
                    print(f"     - {ptype}: {count:,} samples")
                
                # 只保留trt_cp类型
                dos_mask = row_meta['pert_type'] == 'trt_cp'
                n_removed = (~dos_mask).sum()
                
                print(f"\n  ✓ Keeping only 'trt_cp' perturbations")
                print(f"  Removed {n_removed:,} non-trt_cp observations")
            else:
                # 降级方案：如果没有pert_type，使用更保守的BRD-K规则
                print(f"   ⚠️ 'pert_type' not found, using fallback method")
                # 只过滤明确标记为DOS的
                dos_mask = ~row_meta['pert_id'].str.contains('DOS', case=False, na=False)
                n_removed = (~dos_mask).sum()
                print(f"  Removed {n_removed:,} DOS observations")
            
            valid_mask &= dos_mask
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        # ========== Filter 2: Minimum observations ==========
        print(f"\n{'='*80}")
        print(f"FILTER 2: Remove compounds with <{min_observations_per_compound} observations")
        print(f"{'='*80}")
        
        valid_meta = row_meta[valid_mask]
        obs_counts = valid_meta.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations_per_compound].index
        
        print(f"  Compounds with ≥{min_observations_per_compound} observations: "
              f"{len(valid_perts):,}/{obs_counts.nunique():,}")
        
        obs_mask = row_meta['pert_id'].isin(valid_perts)
        valid_mask &= obs_mask
        
        print(f"  Remaining samples: {valid_mask.sum():,}")
        print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        del valid_meta, obs_counts, obs_mask
        gc.collect()
            
        # ========== Filter 3: Cosine similarity ==========

        print(f"\n{'='*80}")
        print(f"FILTER 3: Remove observations with cosine similarity <{min_replicate_similarity}")
        print(f"{'='*80}")
        
        valid_indices = np.where(valid_mask)[0]
        valid_matrix = matrix[valid_mask]
        valid_pert_ids = row_meta.loc[valid_mask, 'pert_id'].reset_index(drop=True)
        
        
        nearest_similarities = self.calculate_cosine_similarity_to_nearest_replicate(
            valid_matrix, 
            valid_pert_ids
        )
        
        full_similarities = np.zeros(len(row_meta), dtype=np.float32)
        full_similarities[valid_indices] = nearest_similarities
        
        sim_mask = (full_similarities >= min_replicate_similarity) | (~valid_mask)
        n_removed_sim = (~sim_mask & valid_mask).sum()
        valid_mask &= sim_mask
        
        print(f"  Removed {n_removed_sim:,} low-similarity observations")
        print(f"  Remaining samples: {valid_mask.sum():,}")
        
        del valid_matrix, valid_pert_ids, nearest_similarities, full_similarities, sim_mask
        gc.collect()
        
        # ========== Filter 4: Dose selection (修正版 + 详细统计) ==========
        print(f"\n{'='*80}")
        print(f"FILTER 4: Select most frequent dose in range {dose_range[0]}-{dose_range[1]} µM")
        print(f"         (Improved: Using intelligent dose binning)")
        print(f"{'='*80}")

        if 'pert_dose' in row_meta.columns:
            
            # ===== Step 1: 解析剂量值 =====
            print(f"  Step 1/6: Parsing dose values...")
            dose_str = row_meta['pert_dose'].astype(str)
            row_meta['dose_value'] = pd.to_numeric(
                dose_str.str.replace(r'[^\d.]', '', regex=True), 
                errors='coerce'
            )
            row_meta['dose_unit'] = dose_str.str.extract(r'([a-zA-Z]+)', expand=False).str.lower()
            
            valid_dose_count = row_meta['dose_value'].notna().sum()
            invalid_dose_count = row_meta['dose_value'].isna().sum()
            print(f"  ✓ Successfully parsed: {valid_dose_count:,} entries")
            print(f"  ⚠️  Missing/invalid doses: {invalid_dose_count:,} entries")
            
            # ===== Step 2: 转换单位到 µM =====
            print(f"\n  Step 2/6: Converting to µM...")
            dose_value = row_meta['dose_value'].values
            dose_unit = row_meta['dose_unit'].values
            
            conditions = [
                dose_unit == 'nm',
                dose_unit == 'mm'
            ]
            choices = [
                dose_value / 1000,
                dose_value * 1000
            ]
            row_meta['dose_uM'] = np.select(conditions, choices, default=dose_value)
            
            print(f"  Dose unit distribution:")
            unit_counts = row_meta['dose_unit'].value_counts()
            for unit, count in unit_counts.head(5).items():
                print(f"    - {unit}: {count:,} samples")
            
            # ===== Step 3: 智能剂量分组 (核心改进) =====
            print(f"\n  Step 3/6: Applying intelligent dose binning...")
            print(f"            (Grouping similar doses like 4.99, 5.00, 5.01 → 5.0)")
            
            # 使用混合方法进行剂量标准化
            row_meta['dose_standardized'] = assign_standard_dose(
                row_meta['dose_uM'].values,
                method='hybrid',
                tolerance_pct=0.1,  # 10% 容差
                round_precision=0.5
            )
            
            # 统计分组效果
            original_unique_doses = row_meta.loc[valid_mask, 'dose_uM'].nunique()
            standardized_unique_doses = row_meta.loc[valid_mask, 'dose_standardized'].nunique()
            print(f"  ✓ Original unique doses: {original_unique_doses:,}")
            print(f"  ✓ Standardized unique doses: {standardized_unique_doses:,}")
            print(f"  ✓ Compression ratio: {original_unique_doses/standardized_unique_doses:.1f}x")
            
            # 展示分组示例
            print(f"\n  📋 Dose binning examples:")
            sample_doses = [4.99, 5.00, 5.01, 5.02, 10.0, 9.98, 10.02, 3.33, 3.3333, 1.11, 1.1111]
            for d in sample_doses:
                std_d = assign_standard_dose([d], method='hybrid')[0]
                print(f"    {d:.4f} µM → {std_d:.4f} µM")
            
            # ===== Step 4: 剂量分布统计 =====
            print(f"\n  Step 4/6: Dose distribution analysis...")
            
            valid_doses = row_meta.loc[valid_mask & (~pd.isna(row_meta['dose_uM'])), 'dose_uM']
            valid_doses_std = row_meta.loc[valid_mask & (~pd.isna(row_meta['dose_standardized'])), 'dose_standardized']
            
            if len(valid_doses) > 0:
                print(f"\n  📊 Dose statistics (before filtering):")
                print(f"    Total samples with valid dose: {len(valid_doses):,}")
                print(f"    Original dose range: {valid_doses.min():.4f} - {valid_doses.max():.2f} µM")
                print(f"    Mean dose: {valid_doses.mean():.4f} µM")
                print(f"    Median dose: {valid_doses.median():.4f} µM")
                
                # 剂量分布（按区间）
                dose_bins = [0, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 500, np.inf]
                dose_labels = ['<0.1', '0.1-0.5', '0.5-1', '1-2', '2-5', '5-10', 
                            '10-20', '20-50', '50-100', '100-500', '>500']
                dose_binned = pd.cut(valid_doses, bins=dose_bins, labels=dose_labels, include_lowest=True)
                dose_dist = dose_binned.value_counts().sort_index()
                
                print(f"\n  Dose distribution by range:")
                for dose_range_label, count in dose_dist.items():
                    pct = count / len(valid_doses) * 100
                    print(f"    {dose_range_label:>10} µM: {count:>8,} ({pct:>5.1f}%)")
                
                # 统计最常见的标准化剂量值
                print(f"\n  Top 20 most frequent STANDARDIZED doses:")
                top_doses_std = valid_doses_std.value_counts().head(20)
                for dose_val, count in top_doses_std.items():
                    pct = count / len(valid_doses_std) * 100
                    print(f"    {dose_val:>8.4f} µM: {count:>8,} samples ({pct:>5.2f}%)")
            
            # ===== Step 5: 应用剂量范围过滤 =====
            print(f"\n  Step 5/6: Filtering dose range {dose_range[0]}-{dose_range[1]} µM...")
            dose_range_mask = (
                (row_meta['dose_uM'] >= dose_range[0]) & 
                (row_meta['dose_uM'] <= dose_range[1]) &
                (~pd.isna(row_meta['dose_uM']))
            )
            
            n_in_range = (valid_mask & dose_range_mask).sum()
            n_out_range = (valid_mask & ~dose_range_mask).sum()
            print(f"  ✓ Samples in target range: {n_in_range:,}")
            print(f"  ✓ Samples outside range: {n_out_range:,}")
            
            n_before_dose = valid_mask.sum()
            valid_mask &= dose_range_mask
            n_removed_range = n_before_dose - valid_mask.sum()
            print(f"  ✓ Removed {n_removed_range:,} samples outside dose range")
            
            # ===== Step 6: 基于标准化剂量找最常见剂量 =====
            print(f"\n  Step 6/6: Finding most frequent STANDARDIZED dose per compound...")
            valid_meta = row_meta[valid_mask].copy()
            unique_compounds = valid_meta['pert_id'].nunique()
            print(f"  Processing {unique_compounds:,} compounds...")
            
            # 使用标准化后的剂量来计算最常见剂量
            tqdm.pandas(desc="    Finding modal doses (standardized)")
            most_common_doses_std = (
                valid_meta.groupby('pert_id')['dose_standardized']
                .progress_apply(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
            )
            
            print(f"  ✓ Identified modal doses for {len(most_common_doses_std):,} compounds")
            
            # 统计 modal dose 分布
            print(f"\n  📊 Modal dose distribution (STANDARDIZED):")
            modal_dose_counts_std = most_common_doses_std.value_counts().head(15)
            for dose_val, count in modal_dose_counts_std.items():
                pct = count / len(most_common_doses_std) * 100
                print(f"    {dose_val:>8.4f} µM: {count:>5,} compounds ({pct:>5.2f}%)")
            
            # 映射到所有行
            row_meta['most_common_dose_std'] = row_meta['pert_id'].map(most_common_doses_std)
            
            # 使用标准化剂量进行匹配
            dose_modal_mask = (
                (row_meta['dose_standardized'] == row_meta['most_common_dose_std']) &
                (~pd.isna(row_meta['most_common_dose_std']))
            )
            
            n_before_modal = valid_mask.sum()
            valid_mask &= dose_modal_mask
            n_removed_modal = n_before_modal - valid_mask.sum()
            
            print(f"\n  ✅ Filter 4 Results (with intelligent dose binning):")
            print(f"  ✓ Removed {n_removed_modal:,} samples with non-modal dose")
            print(f"  ✓ Remaining samples: {valid_mask.sum():,}")
            print(f"  ✓ Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            # ========== 可视化部分（优化版） ==========
            if False:
                print(f"\n  📊 Generating dose distribution visualizations...")
                
                viz_dir = Path("E:/科研/Models/drugreflector/visualizations")
                viz_dir.mkdir(exist_ok=True, parents=True)
                
                # ===== 图1: 综合剂量分析 (2x3布局) =====
                fig, axes = plt.subplots(2, 3, figsize=(18, 12))
                
                # 子图1: 原始剂量分布 (对数刻度)
                ax1 = axes[0, 0]
                valid_doses_before = row_meta.loc[
                    row_meta.index[valid_indices] if 'valid_indices' in locals() else slice(None), 
                    'dose_uM'
                ].dropna()
                
                if len(valid_doses_before) > 0:
                    log_bins = np.logspace(np.log10(0.001), np.log10(1000), 50)
                    ax1.hist(valid_doses_before, bins=log_bins, edgecolor='black', alpha=0.7, color='steelblue')
                    ax1.axvline(dose_range[0], color='red', linestyle='--', linewidth=2, 
                            label=f'Range: {dose_range[0]}-{dose_range[1]} µM')
                    ax1.axvline(dose_range[1], color='red', linestyle='--', linewidth=2)
                    ax1.set_xscale('log')
                    ax1.set_xlabel('Dose (µM, log scale)', fontsize=11)
                    ax1.set_ylabel('Number of samples', fontsize=11)
                    ax1.set_title('Original Dose Distribution\n(Before Filtering)', fontsize=12, fontweight='bold')
                    ax1.legend()
                    ax1.grid(True, alpha=0.3)
                
                # 子图2: 标准化后的剂量分布
                ax2 = axes[0, 1]
                valid_doses_std_before = row_meta.loc[
                    row_meta.index[valid_indices] if 'valid_indices' in locals() else slice(None), 
                    'dose_standardized'
                ].dropna()
                
                if len(valid_doses_std_before) > 0:
                    log_bins = np.logspace(np.log10(0.001), np.log10(1000), 50)
                    ax2.hist(valid_doses_std_before, bins=log_bins, edgecolor='black', alpha=0.7, color='darkorange')
                    ax2.axvline(dose_range[0], color='red', linestyle='--', linewidth=2)
                    ax2.axvline(dose_range[1], color='red', linestyle='--', linewidth=2)
                    ax2.set_xscale('log')
                    ax2.set_xlabel('Standardized Dose (µM, log scale)', fontsize=11)
                    ax2.set_ylabel('Number of samples', fontsize=11)
                    ax2.set_title('Standardized Dose Distribution\n(After Binning)', fontsize=12, fontweight='bold')
                    ax2.grid(True, alpha=0.3)
                
                # 子图3: 过滤后的最终分布
                ax3 = axes[0, 2]
                final_doses = row_meta.loc[valid_mask, 'dose_uM'].dropna()
                
                if len(final_doses) > 0:
                    log_bins = np.logspace(np.log10(dose_range[0]), np.log10(dose_range[1]), 30)
                    ax3.hist(final_doses, bins=log_bins, edgecolor='black', alpha=0.7, color='forestgreen')
                    ax3.set_xscale('log')
                    ax3.set_xlabel('Dose (µM, log scale)', fontsize=11)
                    ax3.set_ylabel('Number of samples', fontsize=11)
                    ax3.set_title('Final Dose Distribution\n(After All Filtering)', fontsize=12, fontweight='bold')
                    ax3.grid(True, alpha=0.3)
                
                # 子图4: 原始 vs 标准化剂量对比 (Top 15)
                ax4 = axes[1, 0]
                
                # 原始剂量 top 15
                top_original = valid_doses.value_counts().head(15)
                top_std = valid_doses_std.value_counts().head(15)
                
                x = np.arange(min(15, len(top_original)))
                width = 0.35
                
                bars1 = ax4.bar(x - width/2, top_original.values[:len(x)], width, 
                            label='Original doses', alpha=0.8, color='steelblue', edgecolor='black')
                bars2 = ax4.bar(x + width/2, top_std.values[:len(x)], width, 
                            label='Standardized doses', alpha=0.8, color='darkorange', edgecolor='black')
                
                ax4.set_ylabel('Sample count', fontsize=11)
                ax4.set_title('Top 15 Doses: Original vs Standardized', fontsize=12, fontweight='bold')
                ax4.set_xticks(x)
                
                # 创建混合标签
                labels = []
                for i in range(len(x)):
                    orig_label = f'{top_original.index[i]:.2f}' if i < len(top_original) else ''
                    std_label = f'{top_std.index[i]:.2f}' if i < len(top_std) else ''
                    labels.append(f'{orig_label}\n({std_label})')
                ax4.set_xticklabels(labels, fontsize=8, rotation=45, ha='right')
                ax4.legend()
                ax4.grid(axis='y', alpha=0.3)
                
                # 子图5: Modal dose 分布 (标准化后)
                ax5 = axes[1, 1]
                modal_doses_final = most_common_doses_std.value_counts().head(20)
                
                if len(modal_doses_final) > 0:
                    bars = ax5.barh(range(len(modal_doses_final)), modal_doses_final.values, 
                                color='mediumpurple', edgecolor='black')
                    ax5.set_yticks(range(len(modal_doses_final)))
                    ax5.set_yticklabels([f'{d:.2f} µM' for d in modal_doses_final.index])
                    ax5.set_xlabel('Number of compounds', fontsize=11)
                    ax5.set_ylabel('Modal Dose (µM)', fontsize=11)
                    ax5.set_title('Top 20 Modal Doses\n(Standardized, Per Compound)', fontsize=12, fontweight='bold')
                    ax5.grid(axis='x', alpha=0.3)
                    
                    for i, (bar, val) in enumerate(zip(bars, modal_doses_final.values)):
                        ax5.text(val, i, f' {val:,}', va='center', fontsize=8)
                
                # 子图6: 剂量标准化效果统计
                ax6 = axes[1, 2]
                
                # 统计5 µM附近的分组效果
                dose_5um_range = (valid_doses >= 4.5) & (valid_doses <= 5.5)
                doses_near_5 = valid_doses[dose_5um_range].value_counts().head(10)
                doses_near_5_std = valid_doses_std[dose_5um_range].value_counts()
                
                # 创建分组效果的条形图
                categories = ['4.5-5.5 µM\n原始唯一值', '4.5-5.5 µM\n标准化后唯一值', 
                            '全局\n原始唯一值', '全局\n标准化后唯一值']
                values = [len(doses_near_5), len(doses_near_5_std),
                        original_unique_doses, standardized_unique_doses]
                colors = ['steelblue', 'darkorange', 'steelblue', 'darkorange']
                
                bars = ax6.bar(categories, values, color=colors, edgecolor='black', alpha=0.8)
                ax6.set_ylabel('Number of unique dose values', fontsize=11)
                ax6.set_title('Dose Binning Effect\n(Unique Doses Reduction)', fontsize=12, fontweight='bold')
                ax6.grid(axis='y', alpha=0.3)
                
                for bar, val in zip(bars, values):
                    ax6.text(bar.get_x() + bar.get_width()/2, val, f'{val:,}', 
                            ha='center', va='bottom', fontsize=10, fontweight='bold')
                
                plt.tight_layout()
                output_path = viz_dir / "dose_distribution_analysis_improved.png"
                plt.savefig(output_path, dpi=300, bbox_inches='tight')
                print(f"  ✓ Saved figure: {output_path}")
                plt.close()
                
                # ===== 图2: 剂量范围对比 =====
                plt.figure(figsize=(14, 8))
                
                dose_bins_viz = [0, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 500, np.inf]
                dose_labels_viz = ['<0.1', '0.1-0.5', '0.5-1', '1-2', '2-5', '5-10', 
                            '10-20', '20-50', '50-100', '100-500', '>500']
                
                # 过滤前（原始）
                doses_before = row_meta.loc[
                    row_meta.index[valid_indices] if 'valid_indices' in locals() else slice(None), 
                    'dose_uM'
                ].dropna()
                binned_before = pd.cut(doses_before, bins=dose_bins_viz, labels=dose_labels_viz, include_lowest=True)
                dist_before = binned_before.value_counts().sort_index()
                
                # 过滤后
                doses_after = row_meta.loc[valid_mask, 'dose_uM'].dropna()
                binned_after = pd.cut(doses_after, bins=dose_bins_viz, labels=dose_labels_viz, include_lowest=True)
                dist_after = binned_after.value_counts().sort_index()
                
                x = np.arange(len(dose_labels_viz))
                width = 0.35
                
                plt.bar(x - width/2, dist_before.values, width, label='Before filtering', 
                        alpha=0.8, color='steelblue', edgecolor='black')
                plt.bar(x + width/2, dist_after.values, width, label='After filtering (with dose binning)', 
                        alpha=0.8, color='forestgreen', edgecolor='black')
                
                plt.xlabel('Dose Range (µM)', fontsize=13, fontweight='bold')
                plt.ylabel('Number of Samples', fontsize=13, fontweight='bold')
                plt.title('Sample Distribution by Dose Range\n(Before vs After Filtering with Intelligent Dose Binning)', 
                        fontsize=14, fontweight='bold')
                plt.xticks(x, dose_labels_viz, rotation=45, ha='right', fontsize=11)
                plt.legend(fontsize=11)
                plt.grid(axis='y', alpha=0.3)
                
                # 添加数值标签
                for i, (v_before, v_after) in enumerate(zip(dist_before.values, dist_after.values)):
                    if v_before > 0:
                        plt.text(i - width/2, v_before, f'{v_before:,}', ha='center', va='bottom', fontsize=8, rotation=90)
                    if v_after > 0:
                        plt.text(i + width/2, v_after, f'{v_after:,}', ha='center', va='bottom', fontsize=8, rotation=90)
                
                plt.tight_layout()
                output_path2 = viz_dir / "dose_range_comparison_improved.png"
                plt.savefig(output_path2, dpi=300, bbox_inches='tight')
                print(f"  ✓ Saved figure: {output_path2}")
                plt.close()
                
                # ===== 图3: 剂量分组详细分析 (展示5µM附近的聚合效果) =====
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                
                # 子图1: 5 µM 附近的原始剂量分布
                ax1 = axes[0]
                dose_5um_mask = (valid_doses >= 4.0) & (valid_doses <= 6.0)
                doses_near_5_detail = valid_doses[dose_5um_mask].value_counts().head(20)
                
                if len(doses_near_5_detail) > 0:
                    bars = ax1.barh(range(len(doses_near_5_detail)), doses_near_5_detail.values, 
                                color='steelblue', edgecolor='black', alpha=0.8)
                    ax1.set_yticks(range(len(doses_near_5_detail)))
                    ax1.set_yticklabels([f'{d:.4f}' for d in doses_near_5_detail.index])
                    ax1.set_xlabel('Sample Count', fontsize=11)
                    ax1.set_ylabel('Original Dose (µM)', fontsize=11)
                    ax1.set_title('Original Doses Near 5 µM\n(Before Standardization)', fontsize=12, fontweight='bold')
                    ax1.grid(axis='x', alpha=0.3)
                    
                    for i, val in enumerate(doses_near_5_detail.values):
                        ax1.text(val, i, f' {val:,}', va='center', fontsize=8)
                
                # 子图2: 标准化后的剂量分布
                ax2 = axes[1]
                dose_5um_mask_std = (valid_doses_std >= 4.0) & (valid_doses_std <= 6.0)
                doses_near_5_std_detail = valid_doses_std[dose_5um_mask_std].value_counts().head(20)
                
                if len(doses_near_5_std_detail) > 0:
                    bars = ax2.barh(range(len(doses_near_5_std_detail)), doses_near_5_std_detail.values, 
                                color='darkorange', edgecolor='black', alpha=0.8)
                    ax2.set_yticks(range(len(doses_near_5_std_detail)))
                    ax2.set_yticklabels([f'{d:.4f}' for d in doses_near_5_std_detail.index])
                    ax2.set_xlabel('Sample Count', fontsize=11)
                    ax2.set_ylabel('Standardized Dose (µM)', fontsize=11)
                    ax2.set_title('Standardized Doses Near 5 µM\n(After Intelligent Binning)', fontsize=12, fontweight='bold')
                    ax2.grid(axis='x', alpha=0.3)
                    
                    for i, val in enumerate(doses_near_5_std_detail.values):
                        ax2.text(val, i, f' {val:,}', va='center', fontsize=8)
                
                plt.suptitle('Dose Binning Effect: Grouping Similar Doses Together\n'
                            '(e.g., 4.99, 5.00, 5.01, 5.02 → 5.0 µM)', fontsize=14, fontweight='bold', y=1.02)
                plt.tight_layout()
                output_path3 = viz_dir / "dose_binning_detail.png"
                plt.savefig(output_path3, dpi=300, bbox_inches='tight')
                print(f"  ✓ Saved figure: {output_path3}")
                plt.close()
                
                print(f"\n  ✅ Visualization complete!")
                print(f"     Figures saved to: {viz_dir}")
                print(f"     - dose_distribution_analysis_improved.png")
                print(f"     - dose_range_comparison_improved.png")
                print(f"     - dose_binning_detail.png")
                
                # 清理临时列
                cols_to_drop = ['dose_value', 'dose_unit', 'most_common_dose_std']
                for col in cols_to_drop:
                    if col in row_meta.columns:
                        row_meta.drop(col, axis=1, inplace=True, errors='ignore')
                
                del valid_meta, dose_range_mask, dose_modal_mask, most_common_doses_std
                gc.collect()

            else:
                print(f"  ⚠️  'pert_dose' not found, skipping dose filter")
            
        
        # ========== Filter 5: Timepoint selection (优化) ==========
        print(f"\n{'='*80}")
        print(f"FILTER 5: Keep only measurements at {valid_timepoints} hours")
        print(f"{'='*80}")
        
        if 'pert_time' in row_meta.columns:
            # 转换为数值类型
            row_meta['time_numeric'] = pd.to_numeric(row_meta['pert_time'], errors='coerce')
            
            # 显示时间点分布
            print(f"   Available timepoints (numeric):")
            time_counts = row_meta['time_numeric'].value_counts().head(10)
            for time_val, count in time_counts.items():
                print(f"     - {time_val} hours: {count:,} samples")
            
            # 匹配valid_timepoints中的数值
            time_mask = row_meta['time_numeric'].isin(valid_timepoints)
            
            if time_mask.sum() == 0:
                print(f"  ⚠️  No samples match timepoints {valid_timepoints}")
                print(f"  Skipping timepoint filter...")
            else:
                n_before = valid_mask.sum()
                valid_mask &= time_mask
                n_removed = n_before - valid_mask.sum()
                
                print(f"  ✓ Kept samples at {valid_timepoints} hours")
                print(f"  Removed {n_removed:,} observations (invalid timepoint)")
                print(f"  Remaining samples: {valid_mask.sum():,}")
            
            del time_mask
            gc.collect()
        else:
            print(f"  ⚠️  'pert_time' not found, skipping timepoint filter")


        # ========== Filter 6: Remove cell lines with <200 compounds ==========
        print(f"\n{'='*80}")
        print(f"FILTER 6: Remove cell lines with <{min_compounds_per_cell} compounds")
        print(f"{'='*80}")

        if cell_id_col is not None:
            valid_meta = row_meta[valid_mask]
            
            # 统计每个细胞系的化合物数量
            compounds_per_cell = valid_meta.groupby(cell_id_col)['pert_id'].nunique()
            
            print(f"  Total cell lines before filtering: {len(compounds_per_cell):,}")
            print(f"  Compounds per cell line (mean): {compounds_per_cell.mean():.1f}")
            print(f"  Compounds per cell line (median): {compounds_per_cell.median():.0f}")
            print(f"  Compounds per cell line (min): {compounds_per_cell.min():.0f}")
            print(f"  Compounds per cell line (max): {compounds_per_cell.max():.0f}")
            
            # 筛选出化合物数≥200的细胞系
            valid_cells = compounds_per_cell[compounds_per_cell >= min_compounds_per_cell].index
            
            print(f"\n  Cell lines with ≥{min_compounds_per_cell} compounds: "
                f"{len(valid_cells):,}/{len(compounds_per_cell):,}")
            
            # 显示被过滤掉的细胞系示例
            removed_cells = compounds_per_cell[compounds_per_cell < min_compounds_per_cell]
            if len(removed_cells) > 0:
                print(f"\n  Sample removed cell lines (showing up to 10):")
                for cell, n_compounds in removed_cells.head(10).items():
                    print(f"    - {cell}: {n_compounds} compounds")
            
            # 应用过滤
            cell_filter_mask = row_meta[cell_id_col].isin(valid_cells)
            n_before = valid_mask.sum()
            valid_mask &= cell_filter_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"\n  Removed {n_removed:,} observations from {len(removed_cells):,} cell lines")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            print(f"  Remaining cell lines: {len(valid_cells):,}")
            
            del valid_meta, compounds_per_cell, cell_filter_mask, removed_cells
            gc.collect()
        else:
            print(f"  ⚠️  Cell line column not found, skipping filter")
        
        # ========== Filter 7: Cell line count ==========
        print(f"\n{'='*80}")
        print(f"FILTER 7: Remove compounds in <{min_cell_lines} or >{max_cell_lines} cell lines")
        print(f"{'='*80}")
        
        if cell_id_col is not None:
            valid_meta = row_meta[valid_mask]
            cell_line_counts = valid_meta.groupby('pert_id')[cell_id_col].nunique()
            valid_perts_cell = cell_line_counts[
                (cell_line_counts >= min_cell_lines) & 
                (cell_line_counts <= max_cell_lines)
            ].index
            
            print(f"  Compounds in {min_cell_lines}-{max_cell_lines} cell lines: "
                  f"{len(valid_perts_cell):,}/{len(cell_line_counts):,}")
            
            cell_mask = row_meta['pert_id'].isin(valid_perts_cell)
            n_before = valid_mask.sum()
            valid_mask &= cell_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"  Removed {n_removed:,} observations")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            del valid_meta, cell_line_counts, cell_mask
            gc.collect()
        else:
            print(f"  ⚠️  Cell line column not found, skipping")
        
        # ========== 最终提取数据 ==========
        print(f"\n{'='*80}")
        print(f"✅ FINAL DATASET")
        print(f"{'='*80}")
        
        final_matrix = matrix[valid_mask].copy()
        final_meta = row_meta[valid_mask].reset_index(drop=True)
        
        del matrix
        gc.collect()
        
        print(f"  Extracted {len(final_matrix):,} samples")
        print(f"  Memory usage: {final_matrix.nbytes / (1024**3):.2f} GB")
        
        # 创建标签
        unique_perts = sorted(final_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in final_meta['pert_id']], dtype=np.int32)
        
        final_compounds = len(unique_perts)
        final_samples = len(final_matrix)
        
        if cell_id_col:
            final_cells = final_meta[cell_id_col].nunique()
        else:
            final_cells = 'Unknown'
        
        print(f"\n  Total samples: {final_samples:,}")
        print(f"  Total compounds: {final_compounds:,}")
        print(f"  Cell lines: {final_cells}")
        print(f"  Gene features: {final_matrix.shape[1]}")
        
        compound_obs = final_meta.groupby('pert_id').size()
        print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
        print(f"  Samples per compound (median): {compound_obs.median():.0f}")
        
        n_compounds_100plus = (compound_obs > 100).sum()
        print(f"  Compounds with >100 observations: {n_compounds_100plus:,}")
        
        print(f"\n📊 Comparison with paper (SI page 2):")
        print(f"  Paper: 425,242 obs, 9,597 compounds, 52 cell lines")
        print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cell lines")
        
        if initial_compounds > 0:
            retention_rate = (final_compounds / initial_compounds) * 100
            print(f"  Compound retention rate: {retention_rate:.1f}%")
        
        # 基因名
        if (self.gene_info is not None) and hasattr(self, "landmark_col_indices"):
            gi = self.gene_info
            landmark_mask = gi['feature_space'] == 'landmark'
            landmark_geneinfo = gi[landmark_mask]
            gene_names = list(landmark_geneinfo['gene_symbol'].values)
            print(f"  ✓ Using geneinfo_beta.txt for gene names")
        else:
            gene_name_col = None
            for possible_col in ['gene_symbol', 'pr_gene_symbol', 'symbol', 'id']:
                if possible_col in col_meta.columns:
                    gene_name_col = possible_col
                    break
            if gene_name_col is None:
                gene_name_col = col_meta.columns[0]
            gene_names = list(col_meta[gene_name_col].values)
            print(f"  ⚠️ Using '{gene_name_col}' from col_meta for gene names")

        training_data = {
            'X': final_matrix,
            'y': labels,
            'folds': np.zeros(len(final_matrix), dtype=np.int32),
            'sample_meta': final_meta,
            'metadata': final_meta,
            'gene_names': gene_names,
            'compound_names': list(unique_perts),
            'pert_to_idx': pert_to_idx
        }

        gc.collect()
        
        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*80}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*80}")
        
        folds = np.zeros(len(sample_meta), dtype=np.int32)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


def main():
    """主程序 - 优化版"""
    print("=" * 80)
    print("🧬 DRUGREFLECTOR DATA PREPROCESSING - Optimized Version")
    print("   Fixed DOS filtering and timepoint matching")
    print("=" * 80)
    
    data_dir = "E:/科研/Models/drugreflector/datasets/LINCS2020"
    loader = LINCS2020DataLoader(data_dir)
    
    try:
        # Step 1: 加载元数据
        print("\n" + "=" * 80)
        print("STEP 1: Loading metadata")
        print("=" * 80)
        gene_info = loader.load_gene_info()
        cell_info = loader.load_cell_info()
        compound_info = loader.load_compound_info()
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 80)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 80)
        matrix, row_meta, col_meta = loader.load_level4_signatures()
        
        # Step 3: 准备训练数据
        print("\n" + "=" * 80)
        print("STEP 3: Preparing training data")
        print("=" * 80)
        training_data = loader.prepare_training_data(
            min_observations_per_compound=5,
            min_replicate_similarity=0.12,
            dose_range=(1, 20.0),
            valid_timepoints=[6, 24], 
            min_cell_lines=5,
            max_cell_lines=40,
            min_compounds_per_cell=200,
            remove_dos=True
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 80)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 80)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存
        print("\n" + "=" * 80)
        print("STEP 5: Saving processed data")
        print("=" * 80)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True, parents=True)
        
        output_file = output_dir / "training_data_lincs2020_optimized_1212_s.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f, protocol=4)
        
        print(f"✓ Saved successfully!")
        print(f"   File size: {output_file.stat().st_size / (1024**2):.1f} MB")
        
        # 最终摘要
        print("\n" + "=" * 80)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 80)
        print(f"📁 Output: {output_file}")
        print(f"\n📊 Final dataset:")
        print(f"   • Samples: {len(training_data['X']):,}")
        print(f"   • Compounds: {len(training_data['compound_names']):,}")
        print(f"   • Genes: {training_data['X'].shape[1]}")
        print(f"   • Memory: {training_data['X'].nbytes / (1024**3):.2f} GB")
        
        # 与论文对比
        paper_samples = 425242
        paper_compounds = 9597
        our_samples = len(training_data['X'])
        our_compounds = len(training_data['compound_names'])
        
        print(f"\n📈 Comparison with paper:")
        print(f"   Samples: {our_samples:,} / {paper_samples:,} ({our_samples/paper_samples*100:.1f}%)")
        print(f"   Compounds: {our_compounds:,} / {paper_compounds:,} ({our_compounds/paper_compounds*100:.1f}%)")
        
        print(f"\n🎯 Ready for training!")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 80)
        print("❌ ERROR")
        print("=" * 80)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    training_data = main()

🧬 DRUGREFLECTOR DATA PREPROCESSING - Optimized Version
   Fixed DOS filtering and timepoint matching

🔬 LINCS 2020 Data Loader - Optimized Version
Data directory: E:\科研\Models\drugreflector\datasets\LINCS2020
Dataset: Expanded CMap LINCS Resource 2020


STEP 1: Loading metadata
📖 Loading gene information...
   File: geneinfo_beta.txt
   ✓ Loaded 12,328 genes
   ✓ Columns: ['gene_id', 'gene_symbol', 'ensembl_id', 'gene_title', 'gene_type', 'src', 'feature_space']
   ✓ Using 'feature_space' column to identify landmarks
   ✓ Landmark genes: 978
   ✓ Expected: 978

   Landmark column indices (first 10): [2154 2155 2156 2157 2158 2159 2160 2161 2162 2163]
   Sample IDs: ['7319', '9915', '55127', '57178', '9695']
   Sample symbols: ['GNAI1', 'HSPA8', 'TXLNA', 'HOMER2', 'FSD1']

📖 Loading cell information...
   File: cellinfo_beta.txt
   ✓ Loaded 240 cell lines
   ✓ Columns: ['cell_iname', 'cellosaurus_id', 'donor_age', 'donor_age_death', 'donor_disease_age_onset', 'doubling_time', 'growth_me

   Computing: 100%|██████████| 22731/22731 [12:45<00:00, 29.70it/s]  


   ✓ Mean similarity: 0.3843
  Removed 37,549 low-similarity observations
  Remaining samples: 1,739,580

FILTER 4: Select most frequent dose in range 1-20.0 µM
         (Improved: Using intelligent dose binning)
  Step 1/6: Parsing dose values...
  ✓ Successfully parsed: 1,805,392 entries
  ⚠️  Missing/invalid doses: 506 entries

  Step 2/6: Converting to µM...
  Dose unit distribution:
    - nan: 506 samples
    - e: 30 samples

  Step 3/6: Applying intelligent dose binning...
            (Grouping similar doses like 4.99, 5.00, 5.01 → 5.0)
  ✓ Original unique doses: 10,752
  ✓ Standardized unique doses: 1
  ✓ Compression ratio: 10752.0x

  📋 Dose binning examples:
    4.9900 µM → 10.0000 µM
    5.0000 µM → 10.0000 µM
    5.0100 µM → 10.0000 µM
    5.0200 µM → 10.0000 µM
    10.0000 µM → 10.0000 µM
    9.9800 µM → 10.0000 µM
    10.0200 µM → 10.0000 µM
    3.3300 µM → 10.0000 µM
    3.3333 µM → 10.0000 µM
    1.1100 µM → 10.0000 µM
    1.1111 µM → 10.0000 µM

  Step 4/6: Dose distrib

    Finding modal doses (standardized): 100%|██████████| 22484/22484 [00:01<00:00, 11547.30it/s]


  ✓ Identified modal doses for 22,484 compounds

  📊 Modal dose distribution (STANDARDIZED):
     10.0000 µM: 22,484 compounds (100.00%)

  ✅ Filter 4 Results (with intelligent dose binning):
  ✓ Removed 0 samples with non-modal dose
  ✓ Remaining samples: 1,142,310
  ✓ Remaining compounds: 22,484
  ⚠️  'pert_dose' not found, skipping dose filter

FILTER 5: Keep only measurements at [6, 24] hours
   Available timepoints (numeric):
     - 24.0 hours: 1,338,831 samples
     - 6.0 hours: 398,943 samples
     - 3.0 hours: 31,925 samples
     - 48.0 hours: 18,614 samples
     - 4.0 hours: 11,372 samples
     - 72.0 hours: 2,308 samples
     - 2.0 hours: 1,135 samples
     - 12.0 hours: 1,083 samples
     - -666.0 hours: 581 samples
     - 120.0 hours: 367 samples
  ✓ Kept samples at [6, 24] hours
  Removed 33,712 observations (invalid timepoint)
  Remaining samples: 1,108,598

FILTER 6: Remove cell lines with <200 compounds
  Total cell lines before filtering: 221
  Compounds per cell line 

In [2]:
import numpy as np
import pickle
from pathlib import Path
import gc

# ==========================================
# 1. 定义转换函数 (Z-score -> scGPT Bins)
# ==========================================
def zscore_to_scgpt_bins(z_scores, threshold=1.5, n_bins=51):
    """
    将连续的 Z-scores 转换为 scGPT 预训练模型可理解的离散 Bins (0-50)。
    
    转换逻辑:
    - 背景噪音 (Zero-shot sparsity): |Z| < 1.5 -> Bin 0 (模拟未表达/无扰动)
    - 上调基因 (Up-regulated):       Z > 1.5  -> Bin 30-50 (模拟中高表达)
    - 下调基因 (Down-regulated):     Z < -1.5 -> Bin 1-10  (模拟低表达/抑制)
    """
    print(f"🔄 正在转换矩阵形状: {z_scores.shape}...")
    
    # 1. 初始化全 0 矩阵 (int32格式)
    binned = np.zeros_like(z_scores, dtype=np.int32)
    
    # 2. 处理上调基因 (Z > Threshold) -> 映射到 [30, 50]
    # Z=1.5 对应 Bin 30，Z>=10 对应 Bin 50
    up_mask = z_scores > threshold
    if up_mask.any():
        z_up = z_scores[up_mask]
        # 截断并在 [1.5, 10] 范围内归一化
        z_up_clipped = np.clip(z_up, threshold, 10.0)
        norm_up = (z_up_clipped - threshold) / (10.0 - threshold)
        # 线性映射
        bins_up = 30 + (norm_up * 20)
        binned[up_mask] = np.round(bins_up).astype(np.int32)
    
    # 3. 处理下调基因 (Z < -Threshold) -> 映射到 [1, 10]
    # Z=-1.5 对应 Bin 10，Z<=-10 对应 Bin 1
    # 注意：在 scGPT 语义中，Bin 值越小代表表达量越低
    down_mask = z_scores < -threshold
    if down_mask.any():
        z_down = np.abs(z_scores[down_mask]) # 取绝对值处理
        z_down_clipped = np.clip(z_down, threshold, 10.0)
        norm_down = (z_down_clipped - threshold) / (10.0 - threshold)
        # 线性映射 (反向：Z 绝对值越大 -> Bin 越小)
        bins_down = 10 - (norm_down * 9)
        binned[down_mask] = np.round(bins_down).astype(np.int32)
        
    # 4. 再次确保范围安全
    binned = np.clip(binned, 0, n_bins-1)
    
    # 打印统计信息帮助检查
    n_total = z_scores.size
    n_zeros = (binned == 0).sum()
    print(f"   📊 转换统计:")
    print(f"      - 稀疏化率 (Bin 0): {n_zeros/n_total*100:.1f}% (原 Z-score 绝对值 < {threshold})")
    print(f"      - 模拟高表达 (Bin 30-50): {(binned >= 30).sum()/n_total*100:.1f}%")
    print(f"      - 模拟低表达 (Bin 1-10): {(binned <= 10).sum()/n_total*100:.1f}% (含 Bin 0)")
    
    return binned

# ==========================================
# 2. 执行转换流程
# ==========================================
def main():
    # 📂 路径配置 (请确认你的输入路径)
    input_path = Path("D:\\科研\\Models\\drugreflector\\processed_data\\training_data_lincs2020_optimized_1201_l.pkl")
    output_path = input_path.parent / f"{input_path.stem}_scgpt.pkl"
    
    print(f"{'='*60}")
    print(f"🚀 开始生成 scGPT 专用数据集")
    print(f"   输入文件: {input_path}")
    print(f"{'='*60}\n")
    
    if not input_path.exists():
        print(f"❌ 错误: 找不到文件 {input_path}")
        return

    # 1. 加载数据
    print("📖 正在加载 Pickle 文件 (可能需要几秒钟)...")
    with open(input_path, 'rb') as f:
        data = pickle.load(f)
    
    print(f"   ✓ 加载完成。原始 X 类型: {data['X'].dtype}")
    
    # 2. 转换 X
    # 如果数据是 float，则进行转换；如果是 int 说明可能已经转换过了
    if np.issubdtype(data['X'].dtype, np.floating):
        X_scgpt = zscore_to_scgpt_bins(data['X'], threshold=1.5)
        
        # 🔥 关键步骤：直接替换 X
        print("\n⚡ 正在替换 data['X'] 为 scGPT Bins 格式...")
        data['X'] = X_scgpt
        
        # 释放内存
        gc.collect()
    else:
        print("\n⚠️ 警告: data['X'] 似乎已经是整数格式，跳过转换步骤。")

    # 3. 保存新数据集
    print(f"\n💾 正在保存新数据集到: {output_path}")
    with open(output_path, 'wb') as f:
        pickle.dump(data, f, protocol=4)
        
    print(f"   ✓ 保存成功! 文件大小: {output_path.stat().st_size / (1024**2):.1f} MB")
    print(f"\n✅ 完成! 现在你可以使用 '{output_path.name}' 直接输入到 scGPT 模型了。")
    print(f"   注意: 此时 data['X'] 是 int32 类型 (LongTensor)。")

if __name__ == "__main__":
    main()

🚀 开始生成 scGPT 专用数据集
   输入文件: D:\科研\Models\drugreflector\processed_data\training_data_lincs2020_optimized_1201_l.pkl

📖 正在加载 Pickle 文件 (可能需要几秒钟)...
   ✓ 加载完成。原始 X 类型: float32
🔄 正在转换矩阵形状: (945908, 978)...
   📊 转换统计:
      - 稀疏化率 (Bin 0): 83.6% (原 Z-score 绝对值 < 1.5)
      - 模拟高表达 (Bin 30-50): 9.2%
      - 模拟低表达 (Bin 1-10): 90.8% (含 Bin 0)

⚡ 正在替换 data['X'] 为 scGPT Bins 格式...

💾 正在保存新数据集到: D:\科研\Models\drugreflector\processed_data\training_data_lincs2020_optimized_1201_l_scgpt.pkl
   ✓ 保存成功! 文件大小: 3790.5 MB

✅ 完成! 现在你可以使用 'training_data_lincs2020_optimized_1201_l_scgpt.pkl' 直接输入到 scGPT 模型了。
   注意: 此时 data['X'] 是 int32 类型 (LongTensor)。


In [ ]:
"""
LINCS 2020 Chemical Filters
根据DrugReflector论文SI第2页实现化学过滤
"""

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
import os
warnings.filterwarnings('ignore')

# RDKit imports
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, AllChem, FilterCatalog
    from rdkit.Chem.FilterCatalog import FilterCatalogParams
    RDKIT_AVAILABLE = True
except ImportError:
    print("⚠️  RDKit not installed. Install with: conda install -c conda-forge rdkit")
    RDKIT_AVAILABLE = False


class ChemicalFilter:
    """
    化学过滤器 - 根据DrugReflector SI实现
    
    Filters applied (SI page 2):
    1. Molecular weight: 60-1000 Da (inclusive)
    2. No more than 1 covalent motif (SMARTS-defined)
    3. No more than 9 NIBR structure flags
    4. Pass BRENK criteria
    5. Must not match 30 SMARTS patterns (not disclosed)
    """
    
    def __init__(self, compound_info_path: str, nibr_filter_csv: Optional[str] = None):
        """
        初始化化学过滤器
        
        参数：
            compound_info_path: compoundinfo_beta.txt路径
            nibr_filter_csv: NIBR过滤器CSV文件路径（可选）
        """
        if not RDKIT_AVAILABLE:
            raise ImportError("RDKit is required for chemical filtering")
        
        self.compound_info_path = Path(compound_info_path)
        self.compound_info = None
        self.filtered_compounds = set()
        
        # 初始化BRENK过滤器
        self.brenk_catalog = self._init_brenk_filter()
        
        # 初始化共价基团SMARTS
        self.covalent_smarts = self._init_covalent_smarts()
        
        # 🔥 新增：初始化NIBR过滤器目录
        self.nibr_catalog = self._init_nibr_filter(nibr_filter_csv)
        
        print(f"{'='*80}")
        print(f"🧪 Chemical Filter Initialized")
        print(f"{'='*80}")
        print(f"Based on DrugReflector SI (page 2)")
        print(f"{'='*80}\n")
    
    def _init_brenk_filter(self):
        """初始化BRENK过滤器"""
        params = FilterCatalogParams()
        params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
        return FilterCatalog.FilterCatalog(params)
    
    def _init_covalent_smarts(self) -> List[str]:
        """
        共价基团SMARTS模式
        
        参考：
        - Backman et al. 2019 (ChEMBL structural alerts)
        - Common covalent warheads in drug discovery
        """
        return [
            # Michael acceptors
            'C=CC(=O)',  # α,β-unsaturated carbonyl
            'C=CC(=O)N',  # acrylamide
            'C=CC#N',  # acrylonitrile
            
            # Electrophilic carbonyls
            '[C;!R](=O)Cl',  # acyl chloride
            '[C;!R](=O)O[C;!R](=O)',  # anhydride
            'C(=O)N=[N+]=[N-]',  # acyl azide
            
            # Epoxides and aziridines
            'C1OC1',  # epoxide
            'C1NC1',  # aziridine
            
            # Haloacetamides
            'ClCC(=O)N',  # chloroacetamide
            'BrCC(=O)N',  # bromoacetamide
            
            # Aldehydes (reactive)
            '[CH;!R]=O',  # aldehyde
            
            # Isocyanates/isothiocyanates
            'N=C=O',  # isocyanate
            'N=C=S',  # isothiocyanate
            
            # Sulfonyl halides
            'S(=O)(=O)Cl',  # sulfonyl chloride
            'S(=O)(=O)F',  # sulfonyl fluoride
            
            # Nitriles (activated)
            '[C;!R]#N',  # nitrile (non-aromatic)
            
            # Peroxides
            'OO',  # peroxide
            
            # Beta-lactams (strained)
            'C1(=O)NCC1',  # beta-lactam
            
            # Activated esters
            'C(=O)OC(=O)',  # activated ester
            
            # Quinones
            'C1=CC(=O)C=CC1=O',  # quinone
        ]
    
    def _init_nibr_filter(self, csv_path: Optional[str] = None):
        """
        初始化NIBR过滤器目录（基于官方444个SMARTS）
        
        参数：
            csv_path: NIBR过滤器CSV文件路径（可选）
            
        返回：
            FilterCatalog对象
        """
        # 🔥 确定CSV文件路径
        if csv_path is None:
            # 默认路径：当前脚本目录下的chem_filter文件夹
            try:
                # 尝试使用 __file__（在脚本模式下）
                script_dir = Path(__file__).parent
            except NameError:
                # 在交互式环境中使用当前工作目录
                script_dir = Path(os.getcwd())
            csv_path = script_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
        else:
            csv_path = Path(csv_path)
        
        if not csv_path.exists():
            print(f"⚠️  NIBR filter CSV not found: {csv_path}")
            print(f"   Using fallback: empty NIBR catalog")
            return FilterCatalog.FilterCatalog()
        
        print(f"📖 Loading NIBR filters from: {csv_path.name}")
        
        # 读取CSV并构建FilterCatalog
        try:
            nibr_df = pd.read_csv(csv_path)
            print(f"   ✓ Loaded {len(nibr_df)} NIBR filter patterns")
            
            nibr_catalog = FilterCatalog.FilterCatalog()
            
            n_added = 0
            for idx, row in nibr_df.iterrows():
                try:
                    # 提取参数
                    pattern_name = row['PATTERN_NAME']
                    smarts = row['SMARTS']
                    min_count = 1 if row['MIN_COUNT'] == 0 else int(row['MIN_COUNT'])
                    severity = int(row['SEVERITY_SCORE'])
                    covalent = int(row['COVALENT'])
                    special_mol = int(row['SPECIAL_MOL'])
                    set_name = row['SET_NAME']
                    
                    # 构建过滤器名称（与官方代码一致）
                    filter_name = f"{pattern_name}_min({min_count})__{severity}__{covalent}__{special_mol}"
                    
                    # 创建SMARTS匹配器
                    matcher = FilterCatalog.SmartsMatcher(filter_name, smarts, min_count)
                    
                    # 添加到目录
                    entry = FilterCatalog.FilterCatalogEntry(filter_name, matcher)
                    entry.SetProp('Scope', set_name)
                    entry.SetProp('Severity', str(severity))
                    nibr_catalog.AddEntry(entry)
                    
                    n_added += 1
                    
                except Exception as e:
                    print(f"   ⚠️  Failed to add filter {idx}: {e}")
                    continue
            
            print(f"   ✓ Successfully added {n_added} NIBR filters to catalog")
            return nibr_catalog
            
        except Exception as e:
            print(f"   ❌ Error loading NIBR filters: {e}")
            print(f"   Using fallback: empty NIBR catalog")
            return FilterCatalog.FilterCatalog()
    
    def load_compound_info(self):
        """加载化合物信息"""
        print(f"📖 Loading compound information...")
        
        if not self.compound_info_path.exists():
            raise FileNotFoundError(
                f"Compound info file not found: {self.compound_info_path}"
            )
        
        self.compound_info = pd.read_csv(self.compound_info_path, sep='\t')
        
        print(f"   ✓ Loaded {len(self.compound_info):,} compounds")
        print(f"   ✓ Columns: {list(self.compound_info.columns)}")
        
        # 检查SMILES列
        smiles_col = None
        for col in ['canonical_smiles', 'smiles', 'SMILES']:
            if col in self.compound_info.columns:
                smiles_col = col
                break
        
        if smiles_col is None:
            raise ValueError(
                f"SMILES column not found. Available columns: {list(self.compound_info.columns)}"
            )
        
        self.smiles_col = smiles_col
        print(f"   ✓ Using '{smiles_col}' for molecular structures")
        
        # 检查有效SMILES数量
        valid_smiles = self.compound_info[smiles_col].notna().sum()
        print(f"   ✓ Compounds with valid SMILES: {valid_smiles:,}")
        
        return self.compound_info
    
    def filter_1_molecular_weight(self, min_mw=60, max_mw=1000) -> set:
        """
        Filter 1: 分子量过滤 (60-1000 Da)
        
        参数：
            min_mw: 最小分子量（默认60）
            max_mw: 最大分子量（默认1000）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 1: Molecular Weight ({min_mw}-{max_mw} Da)")
        print(f"{'='*80}")
        
        passed = set()
        failed_mw = []
        no_smiles = 0
        invalid_smiles = 0
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            # 检查SMILES是否存在
            if pd.isna(smiles) or smiles == '':
                no_smiles += 1
                continue
            
            # 解析分子
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                invalid_smiles += 1
                continue
            
            # 计算分子量
            mw = Descriptors.MolWt(mol)
            
            if min_mw <= mw <= max_mw:
                passed.add(pert_id)
            else:
                failed_mw.append((pert_id, mw))
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (out of MW range): {len(failed_mw):,} compounds")
        print(f"  ⚠️  No SMILES: {no_smiles:,} compounds")
        print(f"  ⚠️  Invalid SMILES: {invalid_smiles:,} compounds")
        
        if failed_mw:
            failed_df = pd.DataFrame(failed_mw, columns=['pert_id', 'MW'])
            print(f"\n  MW distribution of failed compounds:")
            print(f"    • Mean: {failed_df['MW'].mean():.1f} Da")
            print(f"    • Min: {failed_df['MW'].min():.1f} Da")
            print(f"    • Max: {failed_df['MW'].max():.1f} Da")
        
        return passed
    
    def filter_2_covalent_motifs(self, max_motifs=1) -> set:
        """
        Filter 2: 共价基团过滤 (≤1个)
        
        参数：
            max_motifs: 允许的最大共价基团数（默认1）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 2: Covalent Motifs (≤{max_motifs})")
        print(f"{'='*80}")
        print(f"Checking {len(self.covalent_smarts)} covalent SMARTS patterns...")
        
        passed = set()
        failed_counts = []
        
        # 预编译SMARTS
        smarts_mols = []
        for smarts in self.covalent_smarts:
            try:
                smarts_mol = Chem.MolFromSmarts(smarts)
                if smarts_mol:
                    smarts_mols.append(smarts_mol)
            except:
                print(f"  ⚠️  Invalid SMARTS: {smarts}")
        
        print(f"  ✓ Successfully compiled {len(smarts_mols)} SMARTS patterns")
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 计数匹配的共价基团
            motif_count = 0
            for smarts_mol in smarts_mols:
                if mol.HasSubstructMatch(smarts_mol):
                    motif_count += 1
            
            if motif_count <= max_motifs:
                passed.add(pert_id)
            else:
                failed_counts.append(motif_count)
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (>{max_motifs} motifs): {len(failed_counts):,} compounds")
        
        if failed_counts:
            print(f"\n  Distribution of failed compounds:")
            print(f"    • Mean motifs: {np.mean(failed_counts):.1f}")
            print(f"    • Max motifs: {max(failed_counts)}")
        
        return passed
    
    def filter_3_nibr_flags(self, max_flags=9) -> set:
        """
        Filter 3: NIBR结构标记 (≤9个)
        
        使用官方NIBR 444个SMARTS过滤器
        参考：Schuffenhauer et al. 2020, J. Med. Chem.
        
        参数：
            max_flags: 允许的最大标记数（默认9，符合SI要求）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 3: NIBR Structure Flags (≤{max_flags})")
        print(f"{'='*80}")
        print(f"Using official NIBR filter catalog (Schuffenhauer et al. 2020)")
        
        if self.nibr_catalog is None or self.nibr_catalog.GetNumEntries() == 0:
            print(f"  ⚠️  NIBR catalog is empty, skipping this filter")
            # 返回所有化合物（相当于不过滤）
            return set(self.compound_info['pert_id'].values)
        
        print(f"  ✓ Catalog contains {self.nibr_catalog.GetNumEntries()} filter patterns")
        
        passed = set()
        failed_counts = []
        flag_distribution = []
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 🔥 使用FilterCatalog获取所有匹配
            matches = self.nibr_catalog.GetMatches(mol)
            
            if len(matches) == 0:
                # 没有匹配任何flag，通过
                flag_count = 0
                passed.add(pert_id)
            else:
                # 🔥 计算flag数量（参考官方代码逻辑）
                # 只统计severity=1的FLAG，severity=2的EXCLUDE单独处理
                severity_scores = []
                for entry in matches:
                    # 提取severity信息
                    desc = entry.GetDescription()
                    # 格式：pattern_name_min(X)__severity__covalent__special_mol
                    parts = desc.split('__')
                    if len(parts) >= 2:
                        severity = int(parts[1])
                        severity_scores.append(severity)
                
                # 🔥 关键逻辑（与官方代码一致）：
                # 如果有severity=2（EXCLUDE），直接标记为10（必须排除）
                if 2 in severity_scores:
                    flag_count = 10  # 超过阈值，必须排除
                else:
                    # 否则，flag_count = severity=1的数量
                    flag_count = sum(1 for s in severity_scores if s == 1)
                
                flag_distribution.append(flag_count)
                
                if flag_count <= max_flags:
                    passed.add(pert_id)
                else:
                    failed_counts.append(flag_count)
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (>{max_flags} flags): {len(failed_counts):,} compounds")
        
        if flag_distribution:
            print(f"\n  Flag distribution (all compounds):")
            print(f"    • Mean flags: {np.mean(flag_distribution):.2f}")
            print(f"    • Median flags: {np.median(flag_distribution):.0f}")
            print(f"    • Max flags: {max(flag_distribution)}")
            
            # 统计各flag数量的分布
            from collections import Counter
            flag_counts = Counter(flag_distribution)
            print(f"\n  Flag count histogram:")
            for count in sorted(flag_counts.keys())[:15]:  # 只显示前15个
                n_compounds = flag_counts[count]
                print(f"    • {count} flags: {n_compounds:,} compounds")
        
        if failed_counts:
            print(f"\n  Failed compounds statistics:")
            print(f"    • Mean flags: {np.mean(failed_counts):.2f}")
            print(f"    • Max flags: {max(failed_counts)}")
        
        return passed
    
    def filter_4_brenk(self) -> set:
        """
        Filter 4: BRENK过滤器
        
        BRENK过滤器识别不良的化学基团和反应性官能团
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 4: BRENK Criteria")
        print(f"{'='*80}")
        
        passed = set()
        failed = []
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 检查BRENK过滤器
            matches = self.brenk_catalog.GetMatches(mol)
            
            if len(matches) == 0:
                passed.add(pert_id)
            else:
                failed.append((pert_id, len(matches)))
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (BRENK violations): {len(failed):,} compounds")
        
        return passed
    
    def apply_all_filters(self, target_compounds: Optional[set] = None) -> set:
        """
        应用所有化学过滤器
        
        返回：
            target_compounds: 需要过滤的化合物集合（如果提供，则只过滤这些化合物）
        """
        print(f"\n{'='*80}")
        print(f"🧪 APPLYING ALL CHEMICAL FILTERS")
        print(f"{'='*80}")
        
        if self.compound_info is None:
            self.load_compound_info()
        
        #  如果指定了目标化合物，只过滤这些
        if target_compounds is not None:
            print(f"\n⚠️  Filtering only {len(target_compounds):,} compounds present in training data")
            # 临时保存原始数据
            original_compound_info = self.compound_info.copy()
            # 只保留目标化合物
            self.compound_info = self.compound_info[
                self.compound_info['pert_id'].isin(target_compounds)
            ]
            print(f"   Compounds to check: {len(self.compound_info):,}")
        
        initial_compounds = len(self.compound_info)
        print(f"Initial compounds: {initial_compounds:,}")
        
        # 应用各个过滤器
        passed_mw = self.filter_1_molecular_weight()
        passed_covalent = self.filter_2_covalent_motifs()
        passed_nibr = self.filter_3_nibr_flags()
        passed_brenk = self.filter_4_brenk()
        
        # 取交集
        passed_all = passed_mw & passed_covalent & passed_nibr & passed_brenk
        
        # 如果是针对目标化合物的过滤，恢复原始数据
        if target_compounds is not None:
            self.compound_info = original_compound_info
        
        # 统计
        print(f"\n{'='*80}")
        print(f"📊 CHEMICAL FILTER SUMMARY")
        print(f"{'='*80}")
        print(f"  Initial compounds: {initial_compounds:,}")
        print(f"  Passed Filter 1 (MW): {len(passed_mw):,}")
        print(f"  Passed Filter 2 (Covalent): {len(passed_covalent):,}")
        print(f"  Passed Filter 3 (NIBR): {len(passed_nibr):,}")
        print(f"  Passed Filter 4 (BRENK): {len(passed_brenk):,}")
        print(f"  {'─'*80}")
        print(f"  ✅ Passed ALL filters: {len(passed_all):,}")
        print(f"  ❌ Removed: {initial_compounds - len(passed_all):,}")
        print(f"  Retention rate: {len(passed_all)/initial_compounds*100:.1f}%")
        
        self.filtered_compounds = passed_all
        return passed_all


def apply_chemical_filters_to_training_data(
    training_data_path: str,
    compound_info_path: str,
    output_path: Optional[str] = None,
    nibr_filter_csv: Optional[str] = None
) -> Dict:
    """
    对训练数据应用化学过滤器
    
    参数：
        training_data_path: 训练数据pkl文件路径
        compound_info_path: compoundinfo_beta.txt路径
        output_path: 输出路径（可选）
        nibr_filter_csv: NIBR过滤器CSV路径（可选）
    
    返回：
        过滤后的训练数据字典
    """
    print(f"\n{'='*80}")
    print(f"🔬 APPLYING CHEMICAL FILTERS TO TRAINING DATA")
    print(f"{'='*80}\n")
    
    # 1. 加载训练数据
    print(f"📖 Loading training data...")
    with open(training_data_path, 'rb') as f:
        training_data = pickle.load(f)
    
    initial_samples = len(training_data['X'])
    initial_compounds = len(training_data['compound_names'])
    
    print(f"   ✓ Loaded: {initial_samples:,} samples, {initial_compounds:,} compounds")
    
    # 获取训练数据中的化合物
    metadata = training_data['sample_meta']
    training_compounds = set(metadata['pert_id'].unique())
    print(f"   ✓ Training data contains: {len(training_compounds):,} unique compounds")
    
    # 2. 应用化学过滤器（只针对训练数据中的化合物）
    filter_obj = ChemicalFilter(compound_info_path, nibr_filter_csv=nibr_filter_csv)  # 🔥 传递参数
    filter_obj.load_compound_info()
    valid_compounds = filter_obj.apply_all_filters(target_compounds=training_compounds)
    
    # 3. 过滤训练数据
    print(f"\n{'='*80}")
    print(f"📊 FILTERING TRAINING DATA")
    print(f"{'='*80}")
    
    # valid_compounds现在是training_compounds的子集
    print(f"  Compounds in training data: {len(training_compounds):,}")
    print(f"  Compounds passed filters: {len(valid_compounds):,}")
    print(f"  Compounds to remove: {len(training_compounds - valid_compounds):,}")
    
    valid_mask = metadata['pert_id'].isin(valid_compounds)
    
    n_removed_samples = (~valid_mask).sum()
    n_removed_compounds = len(training_compounds - valid_compounds)  # 从训练数据中移除的

    print(f"  Removing {n_removed_samples:,} samples from {n_removed_compounds:,} compounds...")
    
    # 创建新的训练数据
    filtered_data = {
        'X': training_data['X'][valid_mask],
        'y': None,  # 需要重新编码
        'folds': training_data['folds'][valid_mask],
        'sample_meta': metadata[valid_mask].reset_index(drop=True),
        'metadata': metadata[valid_mask].reset_index(drop=True),
        'gene_names': training_data['gene_names'],
        'compound_names': None,  # 需要更新
        'pert_to_idx': None  # 需要重新构建
    }
    
    # 重新构建化合物映射和标签
    new_compound_names = sorted(list(valid_compounds))
    new_pert_to_idx = {pert: idx for idx, pert in enumerate(new_compound_names)}
    new_labels = np.array([
        new_pert_to_idx[pert] 
        for pert in filtered_data['sample_meta']['pert_id']
    ], dtype=np.int32)
    
    filtered_data['compound_names'] = new_compound_names
    filtered_data['pert_to_idx'] = new_pert_to_idx
    filtered_data['y'] = new_labels
    
    # 统计
    final_samples = len(filtered_data['X'])
    final_compounds = len(new_compound_names)
    
    if 'cell_iname' in filtered_data['sample_meta'].columns:
        final_cells = filtered_data['sample_meta']['cell_iname'].nunique()
    else:
        final_cells = 'Unknown'
    
    print(f"\n{'='*80}")
    print(f"✅ FILTERING COMPLETE")
    print(f"{'='*80}")
    print(f"  Before chemical filters:")
    print(f"    • Samples: {initial_samples:,}")
    print(f"    • Compounds: {initial_compounds:,}")
    print(f"  After chemical filters:")
    print(f"    • Samples: {final_samples:,} ({final_samples/initial_samples*100:.1f}%)")
    print(f"    • Compounds: {final_compounds:,} ({final_compounds/initial_compounds*100:.1f}%)")
    print(f"    • Cell lines: {final_cells}")
    print(f"  Removed:")
    print(f"    • Samples: {n_removed_samples:,}")
    print(f"    • Compounds: {n_removed_compounds:,}")
    
    # 与论文对比
    paper_samples = 425242
    paper_compounds = 9597
    paper_cells = 52
    
    print(f"\n📊 Comparison with paper (SI page 2):")
    print(f"  Paper: {paper_samples:,} obs, {paper_compounds:,} compounds, {paper_cells} cells")
    print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cells")
    print(f"  Match rate:")
    print(f"    • Samples: {final_samples/paper_samples*100:.1f}%")
    print(f"    • Compounds: {final_compounds/paper_compounds*100:.1f}%")
    if isinstance(final_cells, int):
        print(f"    • Cell lines: {final_cells/paper_cells*100:.1f}%")
    
    # 4. 保存
    if output_path is None:
        input_path = Path(training_data_path)
        output_path = input_path.parent / f"{input_path.stem}_chemfiltered.pkl"
    else:
        output_path = Path(output_path)
    
    print(f"\n💾 Saving filtered data to: {output_path}")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'wb') as f:
        pickle.dump(filtered_data, f, protocol=4)
    
    file_size_mb = output_path.stat().st_size / (1024**2)
    print(f"   ✓ Saved successfully! ({file_size_mb:.1f} MB)")
    
    return filtered_data


def main():
    """主程序 - 应用化学过滤器"""
    
    # 配置路径
    data_dir = "D:/科研/Models/drugreflector/datasets/LINCS2020"
    processed_dir = "D:/科研/Models/drugreflector/processed_data"
    
    training_data_path = Path(processed_dir) / "training_data_lincs2020_optimized_1212_s.pkl"
    compound_info_path = Path(data_dir) / "compoundinfo_beta.txt"
    output_path = Path(processed_dir) / "training_data_lincs2020_chemfiltered_1212_s.pkl"
    
    # NIBR过滤器CSV路径
    # CSV文件在当前脚本同级的chem_filter文件夹下
    try:
        # 尝试使用 __file__（在脚本模式下）
        script_dir = Path(__file__).parent
    except NameError:
        # 在交互式环境中使用当前工作目录
        script_dir = Path(os.getcwd())
    nibr_csv_path = script_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
    
    # 检查文件
    if not training_data_path.exists():
        print(f"❌ Training data not found: {training_data_path}")
        print(f"   Please run data preprocessing first!")
        return
    
    if not compound_info_path.exists():
        print(f"❌ Compound info not found: {compound_info_path}")
        return
    
    # 检查NIBR CSV文件
    if not nibr_csv_path.exists():
        print(f"⚠️  NIBR filter CSV not found: {nibr_csv_path}")
        print(f"   Will use fallback NIBR filters (less accurate)")
        nibr_csv_path = None
        
    # 应用过滤器
    try:
        filtered_data = apply_chemical_filters_to_training_data(
            training_data_path=str(training_data_path),
            compound_info_path=str(compound_info_path),
            output_path=str(output_path)
        )
        
        print(f"\n{'='*80}")
        print(f"✅ CHEMICAL FILTERING COMPLETE!")
        print(f"{'='*80}")
        print(f"📁 Final output: {output_path}")
        print(f"🎯 Ready for training!")
        
        return filtered_data
        
    except Exception as e:
        print(f"\n{'='*80}")
        print(f"❌ ERROR")
        print(f"{'='*80}")
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    filtered_data = main()


🔬 APPLYING CHEMICAL FILTERS TO TRAINING DATA

📖 Loading training data...
   ✓ Loaded: 904,569 samples, 11,116 compounds
   ✓ Training data contains: 11,116 unique compounds
📖 Loading NIBR filters from: SubstructureFilter_HitTriaging_wPubChemExamples.csv
   ✓ Loaded 444 NIBR filter patterns
   ✓ Successfully added 444 NIBR filters to catalog
🧪 Chemical Filter Initialized
Based on DrugReflector SI (page 2)

📖 Loading compound information...
   ✓ Loaded 39,321 compounds
   ✓ Columns: ['pert_id', 'cmap_name', 'target', 'moa', 'canonical_smiles', 'inchi_key', 'compound_aliases']
   ✓ Using 'canonical_smiles' for molecular structures
   ✓ Compounds with valid SMILES: 33,531

🧪 APPLYING ALL CHEMICAL FILTERS

⚠️  Filtering only 11,116 compounds present in training data
   Compounds to check: 15,449
Initial compounds: 15,449

CHEMICAL FILTER 1: Molecular Weight (60-1000 Da)


[11:26:25] SMILES Parse Error: syntax error while parsing: restricted
[11:26:25] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:25] SMILES Parse Error: syntax error while parsing: restricted
[11:26:25] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:25] SMILES Parse Error: syntax error while parsing: restricted
[11:26:25] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:26] SMILES Parse Error: syntax error while parsing: restricted
[11:26:26] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:26] SMILES Parse Error: syntax error while parsing: restricted
[11:26:26] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:26] SMILES Parse Error: syntax error while parsing: restricted
[11:26:26] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:26] SMILES Parse Error: syntax er

  ✓ Passed: 10,757 compounds
  ✗ Failed (out of MW range): 65 compounds
  ⚠️  No SMILES: 297 compounds
  ⚠️  Invalid SMILES: 16 compounds

  MW distribution of failed compounds:
    • Mean: 1402.8 Da
    • Min: 59.1 Da
    • Max: 2430.9 Da

CHEMICAL FILTER 2: Covalent Motifs (≤1)
Checking 20 covalent SMARTS patterns...
  ✓ Successfully compiled 20 SMARTS patterns


[11:26:27] SMILES Parse Error: syntax error while parsing: restricted
[11:26:27] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:27] SMILES Parse Error: syntax error while parsing: restricted
[11:26:27] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:27] SMILES Parse Error: syntax error while parsing: restricted
[11:26:27] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:29] SMILES Parse Error: syntax error while parsing: restricted
[11:26:29] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:29] SMILES Parse Error: syntax error while parsing: restricted
[11:26:29] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:29] SMILES Parse Error: syntax error while parsing: restricted
[11:26:29] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:29] SMILES Parse Error: syntax er

  ✓ Passed: 10,463 compounds
  ✗ Failed (>1 motifs): 591 compounds

  Distribution of failed compounds:
    • Mean motifs: 2.2
    • Max motifs: 4

CHEMICAL FILTER 3: NIBR Structure Flags (≤9)
Using official NIBR filter catalog (Schuffenhauer et al. 2020)
  ✓ Catalog contains 444 filter patterns


[11:26:31] SMILES Parse Error: syntax error while parsing: restricted
[11:26:31] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:31] SMILES Parse Error: syntax error while parsing: restricted
[11:26:31] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:32] SMILES Parse Error: syntax error while parsing: restricted
[11:26:32] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:54] SMILES Parse Error: syntax error while parsing: restricted
[11:26:54] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:54] SMILES Parse Error: syntax error while parsing: restricted
[11:26:54] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:54] SMILES Parse Error: syntax error while parsing: restricted
[11:26:54] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:26:54] SMILES Parse Error: syntax er

  ✓ Passed: 9,737 compounds
  ✗ Failed (>9 flags): 1,910 compounds

  Flag distribution (all compounds):
    • Mean flags: 2.57
    • Median flags: 0
    • Max flags: 10

  Flag count histogram:
    • 0 flags: 4,872 compounds
    • 1 flags: 1,044 compounds
    • 2 flags: 110 compounds
    • 3 flags: 12 compounds
    • 10 flags: 1,910 compounds

  Failed compounds statistics:
    • Mean flags: 10.00
    • Max flags: 10

CHEMICAL FILTER 4: BRENK Criteria


[11:27:18] SMILES Parse Error: syntax error while parsing: restricted
[11:27:18] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:27:18] SMILES Parse Error: syntax error while parsing: restricted
[11:27:18] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:27:18] SMILES Parse Error: syntax error while parsing: restricted
[11:27:18] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:27:22] SMILES Parse Error: syntax error while parsing: restricted
[11:27:22] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:27:22] SMILES Parse Error: syntax error while parsing: restricted
[11:27:22] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:27:22] SMILES Parse Error: syntax error while parsing: restricted
[11:27:22] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:27:22] SMILES Parse Error: syntax er

  ✓ Passed: 7,586 compounds
  ✗ Failed (BRENK violations): 4,958 compounds

📊 CHEMICAL FILTER SUMMARY
  Initial compounds: 15,449
  Passed Filter 1 (MW): 10,757
  Passed Filter 2 (Covalent): 10,463
  Passed Filter 3 (NIBR): 9,737
  Passed Filter 4 (BRENK): 7,586
  ────────────────────────────────────────────────────────────────────────────────
  ✅ Passed ALL filters: 7,191
  ❌ Removed: 8,258
  Retention rate: 46.5%

📊 FILTERING TRAINING DATA
  Compounds in training data: 11,116
  Compounds passed filters: 7,191
  Compounds to remove: 3,925
  Removing 424,860 samples from 3,925 compounds...

✅ FILTERING COMPLETE
  Before chemical filters:
    • Samples: 904,569
    • Compounds: 11,116
  After chemical filters:
    • Samples: 479,709 (53.0%)
    • Compounds: 7,191 (64.7%)
    • Cell lines: 56
  Removed:
    • Samples: 424,860
    • Compounds: 3,925

📊 Comparison with paper (SI page 2):
  Paper: 425,242 obs, 9,597 compounds, 52 cells
  Ours:  479,709 obs, 7,191 compounds, 56 cells
  Match

In [7]:
"""
LINCS 2020 Chemical Filters
根据DrugReflector论文SI第2页实现化学过滤
"""

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
import os
warnings.filterwarnings('ignore')

# RDKit imports
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, AllChem, FilterCatalog
    from rdkit.Chem.FilterCatalog import FilterCatalogParams
    RDKIT_AVAILABLE = True
except ImportError:
    print("⚠️  RDKit not installed. Install with: conda install -c conda-forge rdkit")
    RDKIT_AVAILABLE = False


class ChemicalFilter:
    """
    化学过滤器 - 根据DrugReflector SI实现
    
    Filters applied (SI page 2):
    1. Molecular weight: 60-1000 Da (inclusive)
    2. No more than 1 covalent motif (SMARTS-defined)
    3. No more than 9 NIBR structure flags
    4. Pass BRENK criteria
    5. Must not match 30 SMARTS patterns (not disclosed)
    """
    
    def __init__(self, compound_info_path: str, nibr_filter_csv: Optional[str] = None):
        """
        初始化化学过滤器
        
        参数：
            compound_info_path: compoundinfo_beta.txt路径
            nibr_filter_csv: NIBR过滤器CSV文件路径（可选）
        """
        if not RDKIT_AVAILABLE:
            raise ImportError("RDKit is required for chemical filtering")
        
        self.compound_info_path = Path(compound_info_path)
        self.compound_info = None
        self.filtered_compounds = set()
        
        # 初始化BRENK过滤器
        self.brenk_catalog = self._init_brenk_filter()
        
        # 初始化共价基团SMARTS
        self.covalent_smarts = self._init_covalent_smarts()
        
        # 🔥 新增：初始化NIBR过滤器目录
        self.nibr_catalog = self._init_nibr_filter(nibr_filter_csv)
        
        print(f"{'='*80}")
        print(f"🧪 Chemical Filter Initialized")
        print(f"{'='*80}")
        print(f"Based on DrugReflector SI (page 2)")
        print(f"{'='*80}\n")
    
    def _init_brenk_filter(self):
        """初始化BRENK过滤器"""
        params = FilterCatalogParams()
        params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
        return FilterCatalog.FilterCatalog(params)
    
    def _init_covalent_smarts(self) -> List[str]:
        """
        共价基团SMARTS模式
        
        参考：
        - Backman et al. 2019 (ChEMBL structural alerts)
        - Common covalent warheads in drug discovery
        """
        return [
            # Michael acceptors
            'C=CC(=O)',  # α,β-unsaturated carbonyl
            'C=CC(=O)N',  # acrylamide
            'C=CC#N',  # acrylonitrile
            
            # Electrophilic carbonyls
            '[C;!R](=O)Cl',  # acyl chloride
            '[C;!R](=O)O[C;!R](=O)',  # anhydride
            'C(=O)N=[N+]=[N-]',  # acyl azide
            
            # Epoxides and aziridines
            'C1OC1',  # epoxide
            'C1NC1',  # aziridine
            
            # Haloacetamides
            'ClCC(=O)N',  # chloroacetamide
            'BrCC(=O)N',  # bromoacetamide
            
            # Aldehydes (reactive)
            '[CH;!R]=O',  # aldehyde
            
            # Isocyanates/isothiocyanates
            'N=C=O',  # isocyanate
            'N=C=S',  # isothiocyanate
            
            # Sulfonyl halides
            'S(=O)(=O)Cl',  # sulfonyl chloride
            'S(=O)(=O)F',  # sulfonyl fluoride
            
            # Nitriles (activated)
            '[C;!R]#N',  # nitrile (non-aromatic)
            
            # Peroxides
            'OO',  # peroxide
            
            # Beta-lactams (strained)
            'C1(=O)NCC1',  # beta-lactam
            
            # Activated esters
            'C(=O)OC(=O)',  # activated ester
            
            # Quinones
            'C1=CC(=O)C=CC1=O',  # quinone
        ]
    
    def _init_nibr_filter(self, csv_path: Optional[str] = None):
        """
        初始化NIBR过滤器目录（基于官方444个SMARTS）
        
        参数：
            csv_path: NIBR过滤器CSV文件路径（可选）
            
        返回：
            FilterCatalog对象
        """
        # 🔥 确定CSV文件路径
        if csv_path is None:
            # 默认路径：当前脚本目录下的chem_filter文件夹
            try:
                # 尝试使用 __file__（在脚本模式下）
                script_dir = Path(__file__).parent
            except NameError:
                # 在交互式环境中使用当前工作目录
                script_dir = Path(os.getcwd())
            csv_path = script_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
        else:
            csv_path = Path(csv_path)
        
        if not csv_path.exists():
            print(f"⚠️  NIBR filter CSV not found: {csv_path}")
            print(f"   Using fallback: empty NIBR catalog")
            return FilterCatalog.FilterCatalog()
        
        print(f"📖 Loading NIBR filters from: {csv_path.name}")
        
        # 读取CSV并构建FilterCatalog
        try:
            nibr_df = pd.read_csv(csv_path)
            print(f"   ✓ Loaded {len(nibr_df)} NIBR filter patterns")
            
            nibr_catalog = FilterCatalog.FilterCatalog()
            
            n_added = 0
            for idx, row in nibr_df.iterrows():
                try:
                    # 提取参数
                    pattern_name = row['PATTERN_NAME']
                    smarts = row['SMARTS']
                    min_count = 1 if row['MIN_COUNT'] == 0 else int(row['MIN_COUNT'])
                    severity = int(row['SEVERITY_SCORE'])
                    covalent = int(row['COVALENT'])
                    special_mol = int(row['SPECIAL_MOL'])
                    set_name = row['SET_NAME']
                    
                    # 构建过滤器名称（与官方代码一致）
                    filter_name = f"{pattern_name}_min({min_count})__{severity}__{covalent}__{special_mol}"
                    
                    # 创建SMARTS匹配器
                    matcher = FilterCatalog.SmartsMatcher(filter_name, smarts, min_count)
                    
                    # 添加到目录
                    entry = FilterCatalog.FilterCatalogEntry(filter_name, matcher)
                    entry.SetProp('Scope', set_name)
                    entry.SetProp('Severity', str(severity))
                    nibr_catalog.AddEntry(entry)
                    
                    n_added += 1
                    
                except Exception as e:
                    print(f"   ⚠️  Failed to add filter {idx}: {e}")
                    continue
            
            print(f"   ✓ Successfully added {n_added} NIBR filters to catalog")
            return nibr_catalog
            
        except Exception as e:
            print(f"   ❌ Error loading NIBR filters: {e}")
            print(f"   Using fallback: empty NIBR catalog")
            return FilterCatalog.FilterCatalog()
    
    def load_compound_info(self):
        """加载化合物信息"""
        print(f"📖 Loading compound information...")
        
        if not self.compound_info_path.exists():
            raise FileNotFoundError(
                f"Compound info file not found: {self.compound_info_path}"
            )
        
        self.compound_info = pd.read_csv(self.compound_info_path, sep='\t')
        
        print(f"   ✓ Loaded {len(self.compound_info):,} compounds")
        print(f"   ✓ Columns: {list(self.compound_info.columns)}")
        
        # 检查SMILES列
        smiles_col = None
        for col in ['canonical_smiles', 'smiles', 'SMILES']:
            if col in self.compound_info.columns:
                smiles_col = col
                break
        
        if smiles_col is None:
            raise ValueError(
                f"SMILES column not found. Available columns: {list(self.compound_info.columns)}"
            )
        
        self.smiles_col = smiles_col
        print(f"   ✓ Using '{smiles_col}' for molecular structures")
        
        # 检查有效SMILES数量
        valid_smiles = self.compound_info[smiles_col].notna().sum()
        print(f"   ✓ Compounds with valid SMILES: {valid_smiles:,}")
        
        return self.compound_info
    
    def filter_1_molecular_weight(self, min_mw=60, max_mw=1000) -> set:
        """
        Filter 1: 分子量过滤 (60-1000 Da)
        
        参数：
            min_mw: 最小分子量（默认60）
            max_mw: 最大分子量（默认1000）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 1: Molecular Weight ({min_mw}-{max_mw} Da)")
        print(f"{'='*80}")
        
        passed = set()
        failed_mw = []
        no_smiles = 0
        invalid_smiles = 0
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            # 检查SMILES是否存在
            if pd.isna(smiles) or smiles == '':
                no_smiles += 1
                continue
            
            # 解析分子
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                invalid_smiles += 1
                continue
            
            # 计算分子量
            mw = Descriptors.MolWt(mol)
            
            if min_mw <= mw <= max_mw:
                passed.add(pert_id)
            else:
                failed_mw.append((pert_id, mw))
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (out of MW range): {len(failed_mw):,} compounds")
        print(f"  ⚠️  No SMILES: {no_smiles:,} compounds")
        print(f"  ⚠️  Invalid SMILES: {invalid_smiles:,} compounds")
        
        if failed_mw:
            failed_df = pd.DataFrame(failed_mw, columns=['pert_id', 'MW'])
            print(f"\n  MW distribution of failed compounds:")
            print(f"    • Mean: {failed_df['MW'].mean():.1f} Da")
            print(f"    • Min: {failed_df['MW'].min():.1f} Da")
            print(f"    • Max: {failed_df['MW'].max():.1f} Da")
        
        return passed
    
    def filter_2_covalent_motifs(self, max_motifs=1) -> set:
        """
        Filter 2: 共价基团过滤 (≤1个)
        
        参数：
            max_motifs: 允许的最大共价基团数（默认1）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 2: Covalent Motifs (≤{max_motifs})")
        print(f"{'='*80}")
        print(f"Checking {len(self.covalent_smarts)} covalent SMARTS patterns...")
        
        passed = set()
        failed_counts = []
        
        # 预编译SMARTS
        smarts_mols = []
        for smarts in self.covalent_smarts:
            try:
                smarts_mol = Chem.MolFromSmarts(smarts)
                if smarts_mol:
                    smarts_mols.append(smarts_mol)
            except:
                print(f"  ⚠️  Invalid SMARTS: {smarts}")
        
        print(f"  ✓ Successfully compiled {len(smarts_mols)} SMARTS patterns")
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 计数匹配的共价基团
            motif_count = 0
            for smarts_mol in smarts_mols:
                if mol.HasSubstructMatch(smarts_mol):
                    motif_count += 1
            
            if motif_count <= max_motifs:
                passed.add(pert_id)
            else:
                failed_counts.append(motif_count)
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (>{max_motifs} motifs): {len(failed_counts):,} compounds")
        
        if failed_counts:
            print(f"\n  Distribution of failed compounds:")
            print(f"    • Mean motifs: {np.mean(failed_counts):.1f}")
            print(f"    • Max motifs: {max(failed_counts)}")
        
        return passed
    
    def filter_3_nibr_flags(self, max_flags=9) -> set:
        """
        Filter 3: NIBR结构标记 (≤9个)
        
        使用官方NIBR 444个SMARTS过滤器
        参考：Schuffenhauer et al. 2020, J. Med. Chem.
        
        参数：
            max_flags: 允许的最大标记数（默认9，符合SI要求）
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 3: NIBR Structure Flags (≤{max_flags})")
        print(f"{'='*80}")
        print(f"Using official NIBR filter catalog (Schuffenhauer et al. 2020)")
        
        if self.nibr_catalog is None or self.nibr_catalog.GetNumEntries() == 0:
            print(f"  ⚠️  NIBR catalog is empty, skipping this filter")
            # 返回所有化合物（相当于不过滤）
            return set(self.compound_info['pert_id'].values)
        
        print(f"  ✓ Catalog contains {self.nibr_catalog.GetNumEntries()} filter patterns")
        
        passed = set()
        failed_counts = []
        flag_distribution = []
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 🔥 使用FilterCatalog获取所有匹配
            matches = self.nibr_catalog.GetMatches(mol)
            
            if len(matches) == 0:
                # 没有匹配任何flag，通过
                flag_count = 0
                passed.add(pert_id)
            else:
                # 🔥 计算flag数量（参考官方代码逻辑）
                # 只统计severity=1的FLAG，severity=2的EXCLUDE单独处理
                severity_scores = []
                for entry in matches:
                    # 提取severity信息
                    desc = entry.GetDescription()
                    # 格式：pattern_name_min(X)__severity__covalent__special_mol
                    parts = desc.split('__')
                    if len(parts) >= 2:
                        severity = int(parts[1])
                        severity_scores.append(severity)
                
                # 🔥 关键逻辑（与官方代码一致）：
                # 如果有severity=2（EXCLUDE），直接标记为10（必须排除）
                if 2 in severity_scores:
                    flag_count = 10  # 超过阈值，必须排除
                else:
                    # 否则，flag_count = severity=1的数量
                    flag_count = sum(1 for s in severity_scores if s == 1)
                
                flag_distribution.append(flag_count)
                
                if flag_count <= max_flags:
                    passed.add(pert_id)
                else:
                    failed_counts.append(flag_count)
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (>{max_flags} flags): {len(failed_counts):,} compounds")
        
        if flag_distribution:
            print(f"\n  Flag distribution (all compounds):")
            print(f"    • Mean flags: {np.mean(flag_distribution):.2f}")
            print(f"    • Median flags: {np.median(flag_distribution):.0f}")
            print(f"    • Max flags: {max(flag_distribution)}")
            
            # 统计各flag数量的分布
            from collections import Counter
            flag_counts = Counter(flag_distribution)
            print(f"\n  Flag count histogram:")
            for count in sorted(flag_counts.keys())[:15]:  # 只显示前15个
                n_compounds = flag_counts[count]
                print(f"    • {count} flags: {n_compounds:,} compounds")
        
        if failed_counts:
            print(f"\n  Failed compounds statistics:")
            print(f"    • Mean flags: {np.mean(failed_counts):.2f}")
            print(f"    • Max flags: {max(failed_counts)}")
        
        return passed
    
    def filter_4_brenk(self) -> set:
        """
        Filter 4: BRENK过滤器
        
        BRENK过滤器识别不良的化学基团和反应性官能团
        
        返回：
            通过过滤的化合物ID集合
        """
        print(f"\n{'='*80}")
        print(f"CHEMICAL FILTER 4: BRENK Criteria")
        print(f"{'='*80}")
        
        passed = set()
        failed = []
        
        for idx, row in self.compound_info.iterrows():
            pert_id = row['pert_id']
            smiles = row[self.smiles_col]
            
            if pd.isna(smiles):
                continue
            
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # 检查BRENK过滤器
            matches = self.brenk_catalog.GetMatches(mol)
            
            if len(matches) == 0:
                passed.add(pert_id)
            else:
                failed.append((pert_id, len(matches)))
        
        print(f"  ✓ Passed: {len(passed):,} compounds")
        print(f"  ✗ Failed (BRENK violations): {len(failed):,} compounds")
        
        return passed
    
    def apply_all_filters(self, target_compounds: Optional[set] = None) -> set:
        """
        应用所有化学过滤器
        
        返回：
            target_compounds: 需要过滤的化合物集合（如果提供，则只过滤这些化合物）
        """
        print(f"\n{'='*80}")
        print(f"🧪 APPLYING ALL CHEMICAL FILTERS")
        print(f"{'='*80}")
        
        if self.compound_info is None:
            self.load_compound_info()
        
        #  如果指定了目标化合物，只过滤这些
        if target_compounds is not None:
            print(f"\n⚠️  Filtering only {len(target_compounds):,} compounds present in training data")
            # 临时保存原始数据
            original_compound_info = self.compound_info.copy()
            # 只保留目标化合物
            self.compound_info = self.compound_info[
                self.compound_info['pert_id'].isin(target_compounds)
            ]
            print(f"   Compounds to check: {len(self.compound_info):,}")
        
        initial_compounds = len(self.compound_info)
        print(f"Initial compounds: {initial_compounds:,}")
        
        # 应用各个过滤器
        passed_mw = self.filter_1_molecular_weight()
        passed_covalent = self.filter_2_covalent_motifs()
        passed_nibr = self.filter_3_nibr_flags()
        passed_brenk = self.filter_4_brenk()
        
        # 取交集
        passed_all = passed_mw & passed_covalent & passed_nibr & passed_brenk
        
        # 如果是针对目标化合物的过滤，恢复原始数据
        if target_compounds is not None:
            self.compound_info = original_compound_info
        
        # 统计
        print(f"\n{'='*80}")
        print(f"📊 CHEMICAL FILTER SUMMARY")
        print(f"{'='*80}")
        print(f"  Initial compounds: {initial_compounds:,}")
        print(f"  Passed Filter 1 (MW): {len(passed_mw):,}")
        print(f"  Passed Filter 2 (Covalent): {len(passed_covalent):,}")
        print(f"  Passed Filter 3 (NIBR): {len(passed_nibr):,}")
        print(f"  Passed Filter 4 (BRENK): {len(passed_brenk):,}")
        print(f"  {'─'*80}")
        print(f"  ✅ Passed ALL filters: {len(passed_all):,}")
        print(f"  ❌ Removed: {initial_compounds - len(passed_all):,}")
        print(f"  Retention rate: {len(passed_all)/initial_compounds*100:.1f}%")
        
        self.filtered_compounds = passed_all
        return passed_all


def apply_chemical_filters_to_training_data(
    training_data_path: str,
    compound_info_path: str,
    output_path: Optional[str] = None,
    nibr_filter_csv: Optional[str] = None
) -> Dict:
    """
    对训练数据应用化学过滤器
    
    参数：
        training_data_path: 训练数据pkl文件路径
        compound_info_path: compoundinfo_beta.txt路径
        output_path: 输出路径（可选）
        nibr_filter_csv: NIBR过滤器CSV路径（可选）
    
    返回：
        过滤后的训练数据字典
    """
    print(f"\n{'='*80}")
    print(f"🔬 APPLYING CHEMICAL FILTERS TO TRAINING DATA")
    print(f"{'='*80}\n")
    
    # 1. 加载训练数据
    print(f"📖 Loading training data...")
    with open(training_data_path, 'rb') as f:
        training_data = pickle.load(f)
    
    initial_samples = len(training_data['X'])
    initial_compounds = len(training_data['compound_names'])
    
    print(f"   ✓ Loaded: {initial_samples:,} samples, {initial_compounds:,} compounds")
    
    # 获取训练数据中的化合物
    metadata = training_data['sample_meta']
    training_compounds = set(metadata['pert_id'].unique())
    print(f"   ✓ Training data contains: {len(training_compounds):,} unique compounds")
    
    # 2. 应用化学过滤器（只针对训练数据中的化合物）
    filter_obj = ChemicalFilter(compound_info_path, nibr_filter_csv=nibr_filter_csv)  # 传递参数
    filter_obj.load_compound_info()
    valid_compounds = filter_obj.apply_all_filters(target_compounds=training_compounds)
    
    # 3. 过滤训练数据
    print(f"\n{'='*80}")
    print(f"📊 FILTERING TRAINING DATA")
    print(f"{'='*80}")
    
    # valid_compounds现在是training_compounds的子集
    print(f"  Compounds in training data: {len(training_compounds):,}")
    print(f"  Compounds passed filters: {len(valid_compounds):,}")
    print(f"  Compounds to remove: {len(training_compounds - valid_compounds):,}")
    
    valid_mask = metadata['pert_id'].isin(valid_compounds)
    
    n_removed_samples = (~valid_mask).sum()
    n_removed_compounds = len(training_compounds - valid_compounds)  # 从训练数据中移除的

    print(f"  Removing {n_removed_samples:,} samples from {n_removed_compounds:,} compounds...")
    
    # 创建新的训练数据
    filtered_data = {
        'X': training_data['X'][valid_mask],
        'y': None,  # 需要重新编码
        'folds': training_data['folds'][valid_mask],
        'sample_meta': metadata[valid_mask].reset_index(drop=True),
        'metadata': metadata[valid_mask].reset_index(drop=True),
        'gene_names': training_data['gene_names'],
        'compound_names': None,  # 需要更新
        'pert_to_idx': None  # 需要重新构建
    }
    
    # 重新构建化合物映射和标签
    new_compound_names = sorted(list(valid_compounds))
    new_pert_to_idx = {pert: idx for idx, pert in enumerate(new_compound_names)}
    new_labels = np.array([
        new_pert_to_idx[pert] 
        for pert in filtered_data['sample_meta']['pert_id']
    ], dtype=np.int32)
    
    filtered_data['compound_names'] = new_compound_names
    filtered_data['pert_to_idx'] = new_pert_to_idx
    filtered_data['y'] = new_labels
    
    # 统计
    final_samples = len(filtered_data['X'])
    final_compounds = len(new_compound_names)
    
    if 'cell_iname' in filtered_data['sample_meta'].columns:
        final_cells = filtered_data['sample_meta']['cell_iname'].nunique()
    else:
        final_cells = 'Unknown'
    
    print(f"\n{'='*80}")
    print(f"✅ FILTERING COMPLETE")
    print(f"{'='*80}")
    print(f"  Before chemical filters:")
    print(f"    • Samples: {initial_samples:,}")
    print(f"    • Compounds: {initial_compounds:,}")
    print(f"  After chemical filters:")
    print(f"    • Samples: {final_samples:,} ({final_samples/initial_samples*100:.1f}%)")
    print(f"    • Compounds: {final_compounds:,} ({final_compounds/initial_compounds*100:.1f}%)")
    print(f"    • Cell lines: {final_cells}")
    print(f"  Removed:")
    print(f"    • Samples: {n_removed_samples:,}")
    print(f"    • Compounds: {n_removed_compounds:,}")
    
    # 与论文对比
    paper_samples = 425242
    paper_compounds = 9597
    paper_cells = 52
    
    print(f"\n📊 Comparison with paper (SI page 2):")
    print(f"  Paper: {paper_samples:,} obs, {paper_compounds:,} compounds, {paper_cells} cells")
    print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cells")
    print(f"  Match rate:")
    print(f"    • Samples: {final_samples/paper_samples*100:.1f}%")
    print(f"    • Compounds: {final_compounds/paper_compounds*100:.1f}%")
    if isinstance(final_cells, int):
        print(f"    • Cell lines: {final_cells/paper_cells*100:.1f}%")
    
    # 4. 保存
    if output_path is None:
        input_path = Path(training_data_path)
        output_path = input_path.parent / f"{input_path.stem}_chemfiltered.pkl"
    else:
        output_path = Path(output_path)
    
    print(f"\n💾 Saving filtered data to: {output_path}")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'wb') as f:
        pickle.dump(filtered_data, f, protocol=4)
    
    file_size_mb = output_path.stat().st_size / (1024**2)
    print(f"   ✓ Saved successfully! ({file_size_mb:.1f} MB)")
    
    return filtered_data


def main():
    """主程序 - 应用化学过滤器"""
    
    # 配置路径
    data_dir = "D:/科研/Models/drugreflector/datasets/LINCS2020"
    processed_dir = "D:/科研/Models/drugreflector/processed_data"
    
    training_data_path = Path(processed_dir) / "training_data_lincs2020_optimized_1201_l_scgpt.pkl"
    compound_info_path = Path(data_dir) / "compoundinfo_beta.txt"
    output_path = Path(processed_dir) / "training_data_lincs2020_chemfiltered_1201_l_scgpt.pkl"
    
    # NIBR过滤器CSV路径
    # CSV文件在当前脚本同级的chem_filter文件夹下
    try:
        # 尝试使用 __file__（在脚本模式下）
        script_dir = Path(__file__).parent
    except NameError:
        # 在交互式环境中使用当前工作目录
        script_dir = Path(os.getcwd())
    nibr_csv_path = script_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
    
    # 检查文件
    if not training_data_path.exists():
        print(f"❌ Training data not found: {training_data_path}")
        print(f"   Please run data preprocessing first!")
        return
    
    if not compound_info_path.exists():
        print(f"❌ Compound info not found: {compound_info_path}")
        return
    
    # 检查NIBR CSV文件
    if not nibr_csv_path.exists():
        print(f"⚠️  NIBR filter CSV not found: {nibr_csv_path}")
        print(f"   Will use fallback NIBR filters (less accurate)")
        nibr_csv_path = None
        
    # 应用过滤器
    try:
        filtered_data = apply_chemical_filters_to_training_data(
            training_data_path=str(training_data_path),
            compound_info_path=str(compound_info_path),
            output_path=str(output_path)
        )
        
        print(f"\n{'='*80}")
        print(f"✅ CHEMICAL FILTERING COMPLETE!")
        print(f"{'='*80}")
        print(f"📁 Final output: {output_path}")
        print(f"🎯 Ready for training!")
        
        return filtered_data
        
    except Exception as e:
        print(f"\n{'='*80}")
        print(f"❌ ERROR")
        print(f"{'='*80}")
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    filtered_data = main()


🔬 APPLYING CHEMICAL FILTERS TO TRAINING DATA

📖 Loading training data...
   ✓ Loaded: 945,908 samples, 14,669 compounds
   ✓ Training data contains: 14,669 unique compounds
📖 Loading NIBR filters from: SubstructureFilter_HitTriaging_wPubChemExamples.csv
   ✓ Loaded 444 NIBR filter patterns
   ✓ Successfully added 444 NIBR filters to catalog
🧪 Chemical Filter Initialized
Based on DrugReflector SI (page 2)

📖 Loading compound information...
   ✓ Loaded 39,321 compounds
   ✓ Columns: ['pert_id', 'cmap_name', 'target', 'moa', 'canonical_smiles', 'inchi_key', 'compound_aliases']
   ✓ Using 'canonical_smiles' for molecular structures
   ✓ Compounds with valid SMILES: 33,531

🧪 APPLYING ALL CHEMICAL FILTERS

⚠️  Filtering only 14,669 compounds present in training data
   Compounds to check: 19,100
Initial compounds: 19,100

CHEMICAL FILTER 1: Molecular Weight (60-1000 Da)


[11:30:50] SMILES Parse Error: syntax error while parsing: restricted
[11:30:50] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:50] SMILES Parse Error: syntax error while parsing: restricted
[11:30:50] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:50] SMILES Parse Error: syntax error while parsing: restricted
[11:30:50] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:52] SMILES Parse Error: syntax error while parsing: restricted
[11:30:52] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:52] SMILES Parse Error: syntax error while parsing: restricted
[11:30:52] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:52] SMILES Parse Error: syntax error while parsing: restricted
[11:30:52] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:52] SMILES Parse Error: syntax er

  ✓ Passed: 14,212 compounds
  ✗ Failed (out of MW range): 66 compounds
  ⚠️  No SMILES: 394 compounds
  ⚠️  Invalid SMILES: 16 compounds

  MW distribution of failed compounds:
    • Mean: 1400.6 Da
    • Min: 59.1 Da
    • Max: 2430.9 Da

CHEMICAL FILTER 2: Covalent Motifs (≤1)
Checking 20 covalent SMARTS patterns...
  ✓ Successfully compiled 20 SMARTS patterns


[11:30:53] SMILES Parse Error: syntax error while parsing: restricted
[11:30:53] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:53] SMILES Parse Error: syntax error while parsing: restricted
[11:30:53] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:53] SMILES Parse Error: syntax error while parsing: restricted
[11:30:53] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:56] SMILES Parse Error: syntax error while parsing: restricted
[11:30:56] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:56] SMILES Parse Error: syntax error while parsing: restricted
[11:30:56] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:56] SMILES Parse Error: syntax error while parsing: restricted
[11:30:56] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:56] SMILES Parse Error: syntax er

  ✓ Passed: 13,908 compounds
  ✗ Failed (>1 motifs): 602 compounds

  Distribution of failed compounds:
    • Mean motifs: 2.2
    • Max motifs: 4

CHEMICAL FILTER 3: NIBR Structure Flags (≤9)
Using official NIBR filter catalog (Schuffenhauer et al. 2020)
  ✓ Catalog contains 444 filter patterns


[11:30:58] SMILES Parse Error: syntax error while parsing: restricted
[11:30:58] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:58] SMILES Parse Error: syntax error while parsing: restricted
[11:30:58] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:30:59] SMILES Parse Error: syntax error while parsing: restricted
[11:30:59] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:31:39] SMILES Parse Error: syntax error while parsing: restricted
[11:31:39] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:31:39] SMILES Parse Error: syntax error while parsing: restricted
[11:31:39] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:31:39] SMILES Parse Error: syntax error while parsing: restricted
[11:31:39] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:31:39] SMILES Parse Error: syntax er

  ✓ Passed: 13,148 compounds
  ✗ Failed (>9 flags): 1,972 compounds

  Flag distribution (all compounds):
    • Mean flags: 2.27
    • Median flags: 0
    • Max flags: 10

  Flag count histogram:
    • 0 flags: 6,103 compounds
    • 1 flags: 1,070 compounds
    • 2 flags: 111 compounds
    • 3 flags: 12 compounds
    • 10 flags: 1,972 compounds

  Failed compounds statistics:
    • Mean flags: 10.00
    • Max flags: 10

CHEMICAL FILTER 4: BRENK Criteria


[11:32:10] SMILES Parse Error: syntax error while parsing: restricted
[11:32:10] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:32:10] SMILES Parse Error: syntax error while parsing: restricted
[11:32:10] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:32:10] SMILES Parse Error: syntax error while parsing: restricted
[11:32:10] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:32:15] SMILES Parse Error: syntax error while parsing: restricted
[11:32:15] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:32:15] SMILES Parse Error: syntax error while parsing: restricted
[11:32:15] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:32:15] SMILES Parse Error: syntax error while parsing: restricted
[11:32:15] SMILES Parse Error: Failed parsing SMILES 'restricted' for input: 'restricted'
[11:32:15] SMILES Parse Error: syntax er

  ✓ Passed: 10,518 compounds
  ✗ Failed (BRENK violations): 5,539 compounds

📊 CHEMICAL FILTER SUMMARY
  Initial compounds: 19,100
  Passed Filter 1 (MW): 14,212
  Passed Filter 2 (Covalent): 13,908
  Passed Filter 3 (NIBR): 13,148
  Passed Filter 4 (BRENK): 10,518
  ────────────────────────────────────────────────────────────────────────────────
  ✅ Passed ALL filters: 10,107
  ❌ Removed: 8,993
  Retention rate: 52.9%

📊 FILTERING TRAINING DATA
  Compounds in training data: 14,669
  Compounds passed filters: 10,107
  Compounds to remove: 4,562
  Removing 436,902 samples from 4,562 compounds...

✅ FILTERING COMPLETE
  Before chemical filters:
    • Samples: 945,908
    • Compounds: 14,669
  After chemical filters:
    • Samples: 509,006 (53.8%)
    • Compounds: 10,107 (68.9%)
    • Cell lines: 57
  Removed:
    • Samples: 436,902
    • Compounds: 4,562

📊 Comparison with paper (SI page 2):
  Paper: 425,242 obs, 9,597 compounds, 52 cells
  Ours:  509,006 obs, 10,107 compounds, 57 cells


In [9]:
"""
scGPT Data Reconstruction - Post-processing Script
用于将已处理的Level 4数据重构为scGPT格式

使用方法:
1. 确保已运行完整的数据预处理流程（生成.pkl文件）
2. 修改main()中的路径配置
3. 运行此脚本
"""

import numpy as np
import pandas as pd
import pickle
import gc
from pathlib import Path
from tqdm import tqdm
from scipy.stats import median_abs_deviation


class Level3Reconstructor:
    """
    scGPT 数据后处理器：
    从已保存的 training_data.pkl 开始
    1. 加载 Level 3 Control 数据并计算板级统计量
    2. 将 Level 4 (Z-score) 重构为 Level 3
    3. 转换为 scGPT Bins
    """
    
    def __init__(self, data_dir):
        """
        Args:
            data_dir: LINCS2020数据目录路径
        """
        self.data_dir = Path(data_dir)
        self.medians = None
        self.mads = None
        self.gene_info = None
        self.inst_info = None
        
    def load_gene_info(self):
        """加载基因信息以获取landmark索引"""
        print("📖 Loading gene information...")
        gene_info_path = self.data_dir / "geneinfo_beta.txt"
        self.gene_info = pd.read_csv(gene_info_path, sep='\t', low_memory=False)
        
        # 找到landmark基因
        landmark_mask = self.gene_info['feature_space'] == 'landmark'
        self.landmark_indices = np.where(landmark_mask)[0]
        
        print(f"   ✓ Loaded {len(self.gene_info)} genes")
        print(f"   ✓ Landmark genes: {landmark_mask.sum()}")
        
        return self.landmark_indices
    
    def load_instance_info(self):
        """加载实例信息以获取板号映射"""
        print("📖 Loading instance information...")
        inst_info_path = self.data_dir / "instinfo_beta.txt"
        self.inst_info = pd.read_csv(inst_info_path, sep='\t', low_memory=False)
        
        print(f"   ✓ Loaded {len(self.inst_info)} instances")
        
        return self.inst_info
    
    def load_control_stats(self, sample_meta):
        """
        步骤 1: 加载 Control 数据并计算板级统计量 (Median, MAD)
        
        Args:
            sample_meta: 训练数据的样本元数据 (DataFrame)
        """
        print(f"\n{'='*80}")
        print(f"🔄 scGPT Reconstruction: Building Plate Background Model")
        print(f"{'='*80}")
        
        # 1. 获取需要的板号列表
        if 'rna_plate' not in sample_meta.columns:
            print("   ⚠️  'rna_plate' not found in sample_meta, loading from instinfo...")
            if self.inst_info is None:
                self.load_instance_info()
            
            # 创建映射
            plate_map = dict(zip(
                self.inst_info['sample_id'], 
                self.inst_info['rna_plate']
            ))
            sample_meta = sample_meta.copy()
            sample_meta['rna_plate'] = sample_meta['sample_id'].map(plate_map)
            
            # 添加调试信息
            print(f"      Samples in training data: {len(sample_meta):,}")
            print(f"      Samples mapped to plates: {sample_meta['rna_plate'].notna().sum():,}")
            print(f"      Unmapped samples: {sample_meta['rna_plate'].isna().sum():,}")

        target_plates = set(sample_meta['rna_plate'].dropna().unique())
        print(f"   🎯 Target plates: {len(target_plates):,}")

        # 显示一些示例板号
        if target_plates:
            print(f"   📋 Sample target plates: {list(target_plates)[:5]}")
                
        # 2. 加载 Level 3 Control 数据
        l3_path = self.data_dir / "level3_beta_ctl_n188708x12328.gctx"
        if not l3_path.exists():
            raise FileNotFoundError(
                f"Missing Level 3 control file: {l3_path}\n"
                f"This file is required for reconstruction. Please download from CMap."
            )
        
        print(f"   📖 Reading Level 3 Control data (landmarks only)...")
        
        # 加载landmark基因索引
        if self.gene_info is None:
            self.load_gene_info()
        
        # 读取GCTX文件 (仅landmark)
        ctl_matrix, ctl_row_meta = self._read_gctx_landmarks(l3_path)
        
        print(f"   ✓ Loaded control matrix: {ctl_matrix.shape}")
        
        # 3. 关联板信息
        print("   🔗 Mapping control samples to plates...")
        if self.inst_info is None:
            self.load_instance_info()

        # 获取control样本的sample_id
        if 'id' in ctl_row_meta.columns:
            ctl_sample_ids = ctl_row_meta['id']
        elif 'sample_id' in ctl_row_meta.columns:
            ctl_sample_ids = ctl_row_meta['sample_id']
        else:
            # 如果都没有，使用索引
            print(f"   ⚠️  No 'id' or 'sample_id' column found in control metadata")
            print(f"   Available columns: {ctl_row_meta.columns.tolist()}")
            ctl_sample_ids = ctl_row_meta.iloc[:, 0]  # 使用第一列

        # 建立快速查找表
        plate_lookup = pd.Series(
            self.inst_info['rna_plate'].values,
            index=self.inst_info['sample_id']
        ).to_dict()

        # 映射板号
        ctl_plates = ctl_sample_ids.map(plate_lookup)

        # ===== 添加调试信息 =====
        print(f"   📊 Plate mapping diagnostics:")
        print(f"      Control samples: {len(ctl_sample_ids):,}")
        print(f"      Mapped to plates: {ctl_plates.notna().sum():,}")
        print(f"      Unmapped (NaN): {ctl_plates.isna().sum():,}")
        print(f"      Unique control plates: {ctl_plates.nunique()}")
        print(f"      Target plates needed: {len(target_plates)}")

        # 显示前几个映射示例
        print(f"\n   🔍 Sample mappings (first 5):")
        for idx in range(min(5, len(ctl_sample_ids))):
            sid = ctl_sample_ids.iloc[idx]
            plate = ctl_plates.iloc[idx]
            print(f"      {sid} → {plate}")

        # 检查重叠
        ctl_unique_plates = set(ctl_plates.dropna().unique())
        overlap = ctl_unique_plates & target_plates
        print(f"\n   🎯 Plate overlap:")
        print(f"      Control plates: {len(ctl_unique_plates)}")
        print(f"      Target plates: {len(target_plates)}")
        print(f"      Overlap: {len(overlap)}")

        if len(overlap) > 0:
            print(f"      Example overlap plates: {list(overlap)[:5]}")

        # 4. 过滤到目标板并计算统计量
        valid_mask = ctl_plates.isin(target_plates)

        if not valid_mask.any():
            # 更详细的错误信息
            print(f"\n   ❌ ERROR: No plate overlap found!")
            print(f"\n   Sample control plates: {list(ctl_unique_plates)[:10]}")
            print(f"   Sample target plates: {list(target_plates)[:10]}")
            
            # 尝试找到相似的板号
            if ctl_unique_plates and target_plates:
                ctl_example = list(ctl_unique_plates)[0]
                target_example = list(target_plates)[0]
                print(f"\n   Example control plate: '{ctl_example}' (type: {type(ctl_example).__name__})")
                print(f"   Example target plate: '{target_example}' (type: {type(target_example).__name__})")
            
            raise ValueError(
                "No overlap between control plates and training data plates!\n"
                "Possible causes:\n"
                "1. Different plate naming formats (check types above)\n"
                "2. Control data is from different batches\n"
                "3. instinfo_beta.txt doesn't match your datasets\n"
                "4. sample_id mapping failed\n\n"
                "Please check the diagnostics above."
            )
        
        print(f"   📊 Computing plate-level statistics (Median & MAD)...")
        print(f"   Valid control samples: {valid_mask.sum():,} / {len(valid_mask):,}")
        
        # 转为DataFrame进行分组计算
        df_ctl = pd.DataFrame(
            ctl_matrix[valid_mask],
            index=ctl_plates[valid_mask]
        )
        
        grouped = df_ctl.groupby(df_ctl.index)
        
        # CMap统计量: Median 和 MAD*1.4826 (robust std)
        self.medians = grouped.median()
        self.mads = grouped.apply(
            lambda x: median_abs_deviation(x, scale='normal', axis=0)
        )
        
        print(f"   ✓ Cached statistics for {len(self.medians)} plates")
        
        # 清理内存
        del ctl_matrix, df_ctl, ctl_row_meta
        gc.collect()
        
        return sample_meta  # 返回带板号的元数据
    
    def _read_gctx_landmarks(self, gctx_path):
        """读取GCTX文件的landmark基因"""
        import h5py
        
        with h5py.File(gctx_path, 'r') as f:
            # ✅ 修正1: 读取 COL (样本) 元数据
            # GCTX标准: 样本信息在 COL，基因信息在 ROW
            sample_meta_dict = {}
            if '/0/META/COL' in f:
                for key in f['/0/META/COL'].keys():
                    sample_meta_dict[key] = f[f'/0/META/COL/{key}'][()].astype(str)
            row_meta = pd.DataFrame(sample_meta_dict)

            # 读取数据
            data_key = '/0/DATA/0/matrix'
            full_matrix = f[data_key]
            matrix_shape = full_matrix.shape
            
            # Landmark 是基因索引，需要排序
            landmark_indices = sorted(self.landmark_indices)
            chunk_size = 10000 
            chunks = []

            print(f"    🔍 Detected matrix shape: {matrix_shape}")

            # === 判断矩阵方向 ===
            # 情况 A: (Genes, Samples) - 标准格式，基因在 dim 0 (12328)
            if matrix_shape[0] == 12328:
                print("    ✓ Format: Genes x Samples (Standard)")
                n_samples = matrix_shape[1]
                
                for i in tqdm(range(0, n_samples, chunk_size), desc="    Loading chunks"):
                    end_idx = min(i + chunk_size, n_samples)
                    # 读取: [Landmarks, Current Samples]
                    chunk = full_matrix[landmark_indices, i:end_idx]
                    chunks.append(chunk)
                
                # 拼接后是 (978, N_Samples)，需要转置为 (N_Samples, 978)
                matrix = np.hstack(chunks).T.astype(np.float32)

            # 情况 B: (Samples, Genes) - 转置格式，基因在 dim 1 (12328)
            # 这就是导致你报错的实际情况
            elif matrix_shape[1] == 12328:
                print("    ✓ Format: Samples x Genes (Transposed)")
                n_samples = matrix_shape[0]
                
                for i in tqdm(range(0, n_samples, chunk_size), desc="    Loading chunks"):
                    end_idx = min(i + chunk_size, n_samples)
                    # 读取: [Current Samples, Landmarks]
                    # 注意：这里我们切片 dim 0 (Samples)，筛选 dim 1 (Genes)
                    chunk = full_matrix[i:end_idx, landmark_indices]
                    chunks.append(chunk)
                
                # 拼接后直接是 (N_Samples, 978)，不需要转置
                matrix = np.vstack(chunks).astype(np.float32)
                
            else:
                raise ValueError(f"Unexpected matrix dimensions: {matrix_shape}. Expected one dimension to be 12328 (genes).")

        return matrix, row_meta
        
    def reconstruct_and_quantize(self, training_data, n_bins=51, noise_threshold=4.0):
        """
        步骤 2: 将 Level 4 重构为 Level 3，然后转换为 scGPT Bins
        
        Args:
            training_data: 训练数据字典 (包含 'X' 和 'sample_meta')
            n_bins: scGPT bin数量 (默认51, 范围0-50)
            noise_threshold: Level 3中低于此值的视为噪音 (设为bin 0)
        
        Returns:
            更新后的 training_data
        """
        if self.medians is None:
            raise ValueError("Please run load_control_stats() first!")
        
        X_zscore = training_data['X']
        sample_meta = training_data['sample_meta']
        
        # 确保有板号
        if 'rna_plate' not in sample_meta.columns:
            sample_meta = self.load_control_stats(sample_meta)
            training_data['sample_meta'] = sample_meta
        
        plates = sample_meta['rna_plate'].values
        
        # 结果矩阵
        X_bins = np.zeros(X_zscore.shape, dtype=np.int32)
        # 🔥 新增: 初始化基线(Ctx)矩阵
        X_ctx_bins = np.zeros(X_zscore.shape, dtype=np.int32)
        
        unique_plates = np.unique(plates)
        
        print(f"\n{'='*80}")
        print(f"⚡ scGPT Reconstruction: Level 4 → Level 3 → Bins")
        print(f"{'='*80}")
        print(f"   Strategy: Z-score → Expression → Rank-based Binning (0-{n_bins-1})")
        print(f"   Noise threshold: < {noise_threshold} → Bin 0")
        
        missing_plates = 0
        total_noise_masked = 0
        
        for plate in tqdm(unique_plates, desc="   Processing Plates"):
            # 1. 获取当前板的样本
            idx = np.where(plates == plate)[0]
            if len(idx) == 0:
                continue
            
            # 2. 获取统计量
            if plate in self.medians.index:
                mu = self.medians.loc[plate].values
                # ✅ 修正：直接获取对象...
                sigma = self.mads.loc[plate]
                if hasattr(sigma, 'values'):
                    sigma = sigma.values
                
                # 3. 重构 Level 3
                # Broad Institute公式: Level3 = (Z * Sigma) + Mu
                X_l3_batch = (X_zscore[idx] * sigma) + mu
                
                # 4. scGPT 离散化: Sample-wise Rank-based Binning
                ranks = np.argsort(np.argsort(X_l3_batch, axis=1), axis=1)
                
                # 映射到 1 ~ (n_bins-1)
                n_genes = X_l3_batch.shape[1]
                batch_bins = np.floor(
                    (ranks / n_genes) * (n_bins - 1)
                ).astype(np.int32) + 1
                
                batch_bins = np.clip(batch_bins, 1, n_bins - 1)
                
                # 5. 噪音掩码 (可选但推荐)
                noise_mask = X_l3_batch < noise_threshold
                batch_bins[noise_mask] = 0
                
                total_noise_masked += noise_mask.sum()
                
                X_bins[idx] = batch_bins
                
                # 新增: 处理 Control (Ctx) 数据
                # mu 是该板的基线表达 (1D array)，我们将其也进行 Binning 并广播
                mu_2d = mu.reshape(1, -1)
                
                # 对基线数据进行同样的 Rank-based Binning
                ctx_ranks = np.argsort(np.argsort(mu_2d, axis=1), axis=1)
                ctx_bins_row = np.floor(
                    (ctx_ranks / n_genes) * (n_bins - 1)
                ).astype(np.int32) + 1
                ctx_bins_row = np.clip(ctx_bins_row, 1, n_bins - 1)
                
                # 基线噪音掩码
                ctx_noise_mask = mu_2d < noise_threshold
                ctx_bins_row[ctx_noise_mask] = 0
                
                # 广播到当前板的所有样本 (复制 N 次)
                X_ctx_bins[idx] = np.tile(ctx_bins_row, (len(idx), 1))
                
            else:
                missing_plates += 1
                # 降级方案: 直接对Z-score分箱
                z_batch = X_zscore[idx]
                # Z在 -2~2 范围内映射到 1~50
                temp_bins = (
                    (np.clip(z_batch, -2, 2) + 2) / 4 * (n_bins - 1)
                ).astype(np.int32) + 1
                temp_bins = np.clip(temp_bins, 1, n_bins - 1)
                X_bins[idx] = temp_bins
                
                # 新增: 缺失板的 Ctx 设为默认值 (Z=0 对应的 bin，约 25)
                mid_bin = int((n_bins - 1) / 2)
                X_ctx_bins[idx] = mid_bin
        
        if missing_plates > 0:
            print(f"   ⚠️  Warning: {missing_plates} plates missing from control data")
            print(f"       Used fallback Z-score binning for those plates")
        
        print(f"   ✓ Noise masking: {total_noise_masked:,} values set to bin 0")
        print(f"   ✓ Bin distribution:")
        
        # 统计bin分布
        unique_bins, counts = np.unique(X_bins, return_counts=True)
        bin_dist = dict(zip(unique_bins, counts))
        
        print(f"       Bin 0 (noise/padding): {bin_dist.get(0, 0):,}")
        print(f"       Bins 1-{n_bins-1} (expression): {X_bins[X_bins > 0].size:,}")
        
        # 更新数据
        print(f"   ✓ Updating training_data...")
        
        # 备份原始Z-score (可选)
        training_data['X_level4_zscore'] = training_data['X']
        training_data['X'] = X_bins
        training_data['X_ctx'] = X_ctx_bins
        
        # 添加重构元数据
        training_data['reconstruction_info'] = {
            'n_bins': n_bins,
            'noise_threshold': noise_threshold,
            'missing_plates': missing_plates,
            'noise_masked_count': int(total_noise_masked)
        }
        
        return training_data


def apply_scgpt_reconstruction(
    training_data_path,
    data_dir,
    output_path=None,
    n_bins=51,
    noise_threshold=4.0
):
    """
    完整的scGPT重构流程
    
    Args:
        training_data_path: 已处理的训练数据pkl文件路径
        data_dir: LINCS2020数据目录
        output_path: 输出路径 (None则自动生成)
        n_bins: bin数量
        noise_threshold: 噪音阈值
    
    Returns:
        重构后的训练数据
    """
    print(f"\n{'='*80}")
    print(f"🧬 scGPT Data Reconstruction - Post-processing")
    print(f"{'='*80}")
    
    # 1. 加载训练数据
    print(f"\n📖 Loading training data...")
    print(f"   File: {training_data_path}")
    
    with open(training_data_path, 'rb') as f:
        training_data = pickle.load(f)
    
    print(f"   ✓ Loaded successfully")
    print(f"   Current data shape: {training_data['X'].shape}")
    print(f"   Current data type: {training_data['X'].dtype}")
    print(f"   Samples: {len(training_data['X']):,}")
    print(f"   Genes: {training_data['X'].shape[1]}")
    
    # 检查是否已经是bins格式
    if training_data['X'].dtype in [np.int32, np.int64]:
        print(f"   ⚠️  Warning: Data appears to be already binned (dtype={training_data['X'].dtype})")
        user_input = input("   Continue anyway? (y/n): ")
        if user_input.lower() != 'y':
            print("   Aborted.")
            return None
    
    # 2. 初始化重构器
    reconstructor = Level3Reconstructor(data_dir)
    
    # 3. 加载control统计量
    sample_meta_with_plates = reconstructor.load_control_stats(
        training_data['sample_meta']
    )
    training_data['sample_meta'] = sample_meta_with_plates
    
    # 4. 执行重构和离散化
    training_data = reconstructor.reconstruct_and_quantize(
        training_data,
        n_bins=n_bins,
        noise_threshold=noise_threshold
    )
    
    # 5. 保存
    if output_path is None:
        input_path = Path(training_data_path)
        output_path = input_path.parent / f"{input_path.stem}_scgpt.pkl"
    else:
        output_path = Path(output_path)
    
    print(f"\n{'='*80}")
    print(f"💾 Saving reconstructed data...")
    print(f"{'='*80}")
    print(f"   Output: {output_path}")
    
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'wb') as f:
        pickle.dump(training_data, f, protocol=4)
    
    file_size_mb = output_path.stat().st_size / (1024**2)
    print(f"   ✓ Saved successfully! ({file_size_mb:.1f} MB)")
    
    # 6. 总结
    print(f"\n{'='*80}")
    print(f"✅ scGPT RECONSTRUCTION COMPLETE")
    print(f"{'='*80}")
    print(f"📊 Summary:")
    print(f"   • Input:  Level 4 (Z-score)")
    print(f"   • Output: scGPT Bins (0-{n_bins-1})")
    print(f"   • Samples: {len(training_data['X']):,}")
    print(f"   • Genes: {training_data['X'].shape[1]}")
    print(f"   • File: {output_path}")
    
    if 'reconstruction_info' in training_data:
        info = training_data['reconstruction_info']
        print(f"\n🔍 Reconstruction Details:")
        print(f"   • Bins: {info['n_bins']}")
        print(f"   • Noise threshold: {info['noise_threshold']}")
        print(f"   • Noise-masked values: {info['noise_masked_count']:,}")
        if info['missing_plates'] > 0:
            print(f"   • Missing plates (fallback): {info['missing_plates']}")
    
    print(f"\n🎯 Ready for scGPT training!")
    
    return training_data


def main():
    """主程序"""
    
    # ============ 配置路径 ============
    # 请根据您的实际路径修改以下配置
    
    # LINCS2020原始数据目录
    data_dir = r"D:/科研/Models/drugreflector/datasets/LINCS2020"
    
    # 已处理的训练数据路径 (从您的预处理流程生成的pkl)
    training_data_path = r"D:/科研/Models/drugreflector/processed_data/training_data_lincs2020_chemfiltered_1201_l.pkl"
    
    # 输出路径 (None则自动生成)
    output_path = None  # 将自动命名为 *_scgpt.pkl
    
    # scGPT参数
    n_bins = 51  # scGPT使用的bin数量 (0-50)
    noise_threshold = 4.0  # Level 3中的噪音阈值
    
    # ============ 检查文件 ============
    if not Path(training_data_path).exists():
        print(f"❌ Training data not found: {training_data_path}")
        print(f"\n💡 Hint:")
        print(f"   1. Make sure you've run the preprocessing notebook first")
        print(f"   2. Check if the path is correct")
        print(f"   3. The file should end with .pkl")
        return None
    
    if not Path(data_dir).exists():
        print(f"❌ Data directory not found: {data_dir}")
        print(f"\n💡 Hint:")
        print(f"   This should be your LINCS2020 download directory")
        print(f"   It should contain files like:")
        print(f"   - level3_beta_ctl_n188708x12328.gctx")
        print(f"   - geneinfo_beta.txt")
        print(f"   - instinfo_beta.txt")
        return None
    
    # ============ 执行重构 ============
    try:
        reconstructed_data = apply_scgpt_reconstruction(
            training_data_path=training_data_path,
            data_dir=data_dir,
            output_path=output_path,
            n_bins=n_bins,
            noise_threshold=noise_threshold
        )
        
        return reconstructed_data
        
    except Exception as e:
        print(f"\n{'='*80}")
        print(f"❌ ERROR")
        print(f"{'='*80}")
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        print(f"\n💡 Common Issues:")
        print(f"   1. Missing level3_beta_ctl file - download from CMap")
        print(f"   2. Memory error - try processing smaller batches")
        print(f"   3. Plate mismatch - check instinfo consistency")
        
        return None


if __name__ == "__main__":
    reconstructed_data = main()


🧬 scGPT Data Reconstruction - Post-processing

📖 Loading training data...
   File: D:/科研/Models/drugreflector/processed_data/training_data_lincs2020_chemfiltered_1201_l.pkl
   ✓ Loaded successfully
   Current data shape: (509006, 978)
   Current data type: float32
   Samples: 509,006
   Genes: 978

🔄 scGPT Reconstruction: Building Plate Background Model
   🎯 Target plates: 4,583
   📋 Sample target plates: ['CRCGN015_MCF10A.TP53.M_24H_X1', 'REP.B025_MCF7_24H_X1', 'REP.B028_MCF7_24H_X1', 'REP.A018_YAPC_24H_X2', 'LJP007_ASC_24H_X3.A2']
   📖 Reading Level 3 Control data (landmarks only)...
📖 Loading gene information...
   ✓ Loaded 12328 genes
   ✓ Landmark genes: 978
    🔍 Detected matrix shape: (188708, 12328)
    ✓ Format: Samples x Genes (Transposed)


    Loading chunks: 100%|██████████| 19/19 [00:03<00:00,  5.90it/s]


   ✓ Loaded control matrix: (188708, 978)
   🔗 Mapping control samples to plates...
📖 Loading instance information...
   ✓ Loaded 3026460 instances
   📊 Plate mapping diagnostics:
      Control samples: 188,708
      Mapped to plates: 188,708
      Unmapped (NaN): 0
      Unique control plates: 8027
      Target plates needed: 4583

   🔍 Sample mappings (first 5):
      ABY001_A375_XH_X1_B15:A03 → ABY001_A375_XH_X1
      ABY001_A375_XH_X1_B15:A04 → ABY001_A375_XH_X1
      ABY001_A375_XH_X1_B15:A05 → ABY001_A375_XH_X1
      ABY001_A375_XH_X1_B15:A06 → ABY001_A375_XH_X1
      ABY001_A375_XH_X1_B15:A07 → ABY001_A375_XH_X1

   🎯 Plate overlap:
      Control plates: 8027
      Target plates: 4583
      Overlap: 4576
      Example overlap plates: ['CRCGN015_MCF10A.TP53.M_24H_X1', 'REP.B025_MCF7_24H_X1', 'REP.B028_MCF7_24H_X1', 'REP.A018_YAPC_24H_X2', 'LJP007_ASC_24H_X3.A2']
   📊 Computing plate-level statistics (Median & MAD)...
   Valid control samples: 89,779 / 188,708
   ✓ Cached statisti

   Processing Plates: 100%|██████████| 4583/4583 [00:59<00:00, 76.57it/s] 


   ⚠️  Warning: 7 plates missing from control data
       Used fallback Z-score binning for those plates
   ✓ Noise masking: 16,378,822 values set to bin 0
   ✓ Bin distribution:
       Bin 0 (noise/padding): 16,378,822
       Bins 1-50 (expression): 481,429,046
   ✓ Updating training_data...

💾 Saving reconstructed data...
   Output: D:\科研\Models\drugreflector\processed_data\training_data_lincs2020_chemfiltered_1201_l_scgpt.pkl
   ✓ Saved successfully! (5955.2 MB)

✅ scGPT RECONSTRUCTION COMPLETE
📊 Summary:
   • Input:  Level 4 (Z-score)
   • Output: scGPT Bins (0-50)
   • Samples: 509,006
   • Genes: 978
   • File: D:\科研\Models\drugreflector\processed_data\training_data_lincs2020_chemfiltered_1201_l_scgpt.pkl

🔍 Reconstruction Details:
   • Bins: 51
   • Noise threshold: 4.0
   • Noise-masked values: 16,378,822
   • Missing plates (fallback): 7

🎯 Ready for scGPT training!


In [1]:
import numpy as np
import pandas as pd
import pickle
import h5py
import gc
from pathlib import Path
from tqdm import tqdm
from scipy.stats import median_abs_deviation

# ================= 配置路径 (参考你的Notebook) =================
DATA_DIR = Path(r"D:/科研/Models/drugreflector/datasets/LINCS2020")
PROCESSED_DIR = Path(r"D:/科研/Models/drugreflector/processed_data")

# 输入文件：经过化学过滤的 Level 4 Z-score 数据 (Float)
INPUT_FILE = PROCESSED_DIR / "training_data_lincs2020_chemfiltered_1201_l.pkl"
# 输出文件：包含 scGPT 基线编码的新数据
OUTPUT_FILE = PROCESSED_DIR / "training_data_lincs2020_chemfiltered_1201_l_dual_head.pkl"

# scGPT 参数
N_BINS = 51  # 0-50
NOISE_THRESHOLD = 4.0  # Level 3 (log2) 的噪音阈值，低于此值设为 Bin 0

def load_and_process_ctx():
    print(f"{'='*80}")
    print(f"🧬 Generating Baseline Context (X_ctx) for Dual-Head Model")
    print(f"{'='*80}")

    # 1. 加载训练数据 (Level 4 Z-scores)
    print(f"\n📖 Loading input data: {INPUT_FILE.name}...")
    with open(INPUT_FILE, 'rb') as f:
        data = pickle.load(f)
    
    # 确保 X 是 float32 (Level 4 Z-scores)
    if not np.issubdtype(data['X'].dtype, np.floating):
        print("⚠️ Warning: Input 'X' is not float. Please ensure you are using the raw Z-score dataset.")
    
    sample_meta = data['sample_meta']
    print(f"   ✓ Samples: {len(sample_meta):,}")
    print(f"   ✓ Genes: {data['X'].shape[1]}")

    # 2. 准备板号 (Plate IDs)
    if 'rna_plate' not in sample_meta.columns:
        print("   ⏳ Mapping sample_id to rna_plate...")
        inst_path = DATA_DIR / "instinfo_beta.txt"
        inst_info = pd.read_csv(inst_path, sep='\t', usecols=['sample_id', 'rna_plate'])
        plate_map = dict(zip(inst_info['sample_id'], inst_info['rna_plate']))
        sample_meta['rna_plate'] = sample_meta['sample_id'].map(plate_map)
    
    target_plates = set(sample_meta['rna_plate'].dropna().unique())
    print(f"   ✓ Target Plates: {len(target_plates):,}")

    # 3. 读取 Level 3 Control 数据 (GCTX)
    l3_path = DATA_DIR / "level3_beta_ctl_n188708x12328.gctx"
    print(f"\n📖 Reading Level 3 Control Data from: {l3_path.name}")
    
    # 获取 Landmark 基因索引
    gene_path = DATA_DIR / "geneinfo_beta.txt"
    gene_info = pd.read_csv(gene_path, sep='\t')
    landmark_indices = sorted(np.where(gene_info['feature_space'] == 'landmark')[0])
    
    # 读取 GCTX (自动处理转置)
    with h5py.File(l3_path, 'r') as f:
        # 读取样本元数据以获取板号
        sample_ids_ctl = f['/0/META/COL/id'][()].astype(str)
        
        # 读取矩阵数据
        ds = f['/0/DATA/0/matrix']
        if ds.shape[0] == 12328: # Genes x Samples
            print("   ✓ Format: Genes x Samples")
            # 这种格式很难按样本切片，必须读整块或转置读取
            # 为了内存安全，我们假设大部分情况是 Samples x Genes (LINCS 2020 L3)
            # 如果确实是 Genes x Samples，建议使用分块读取并转置
            # 这里简化处理：读取所有 landmark 行
            ctl_matrix = ds[landmark_indices, :].T # -> (Samples, Landmarks)
        else: # Samples x Genes (188708 x 12328) - 最常见情况
            print("   ✓ Format: Samples x Genes")
            # 直接切片读取所有行，列取landmark
            ctl_matrix = ds[:, landmark_indices].astype(np.float32)
            
    print(f"   ✓ Loaded Control Matrix: {ctl_matrix.shape}")

    # 4. 计算每个板的基线 (Median)
    print("\n📊 Computing Plate Baselines (Median)...")
    
    # 映射 Control 样本到板号
    inst_path = DATA_DIR / "instinfo_beta.txt"
    # 重新读取以确保完整（如果内存够）
    if 'inst_info' not in locals():
        inst_info = pd.read_csv(inst_path, sep='\t', usecols=['sample_id', 'rna_plate'])
    
    plate_lookup = dict(zip(inst_info['sample_id'], inst_info['rna_plate']))
    ctl_plates = pd.Series(sample_ids_ctl).map(plate_lookup)
    
    # 过滤出需要的板
    valid_mask = ctl_plates.isin(target_plates)
    df_ctl = pd.DataFrame(ctl_matrix[valid_mask], index=ctl_plates[valid_mask])
    
    # GroupBy 计算中位数
    plate_medians = df_ctl.groupby(df_ctl.index).median()
    print(f"   ✓ Computed baselines for {len(plate_medians):,} plates")

    # 5. 生成 X_ctx (Binning)
    print("\n⚡ Generating Binned Context Embeddings...")
    X_ctx = np.zeros(data['X'].shape, dtype=np.int32)
    
    # 预计算：所有板的中位数矩阵
    # 这是一个 (N_samples, N_genes) 的矩阵，每行是对应板的基线
    # 为了速度，我们先对 plate_medians 进行 Binning
    
    print("   👉 Performing Rank-based Binning on Baselines...")
    # Rank-based binning: 0-50
    # 对每个板的基线进行排序和分箱
    pm_values = plate_medians.values # (N_plates, N_genes)
    n_genes = pm_values.shape[1]
    
    # argsort twice to get ranks (0 to n_genes-1)
    ranks = np.argsort(np.argsort(pm_values, axis=1), axis=1)
    
    # Binning formula: floor(rank / n_genes * (n_bins - 1)) + 1
    # Bins: 1 to 50
    binned_medians = np.floor((ranks / n_genes) * (N_BINS - 1)).astype(np.int32) + 1
    binned_medians = np.clip(binned_medians, 1, N_BINS - 1)
    
    # Noise masking: values < threshold -> Bin 0
    noise_mask = pm_values < NOISE_THRESHOLD
    binned_medians[noise_mask] = 0
    
    # 创建板号到 binned vector 的映射
    plate_to_bin = dict(zip(plate_medians.index, binned_medians))
    
    # 填充主矩阵
    # 使用 pandas map 快速填充
    # 如果有板号缺失，默认填充为中间值(25)或0
    default_vec = np.full(n_genes, int((N_BINS-1)/2), dtype=np.int32)
    
    # 获取每个样本对应的板号
    sample_plates = sample_meta['rna_plate'].values
    
    # 填充
    missing_count = 0
    for i, plate in enumerate(tqdm(sample_plates, desc="   Mapping to samples")):
        if plate in plate_to_bin:
            X_ctx[i] = plate_to_bin[plate]
        else:
            X_ctx[i] = default_vec
            missing_count += 1
            
    if missing_count > 0:
        print(f"   ⚠️ Warning: {missing_count} samples had missing plate baselines (filled with default).")

    # 6. 更新并保存
    data['X_ctx'] = X_ctx
    data['metadata']['rna_plate'] = sample_plates # 确保metadata里有板号
    
    print(f"\n💾 Saving to {OUTPUT_FILE}...")
    with open(OUTPUT_FILE, 'wb') as f:
        pickle.dump(data, f, protocol=4)
        
    print(f"✅ Done! Output Dataset Keys: {list(data.keys())}")
    print(f"   X shape (Z-score): {data['X'].shape}, dtype={data['X'].dtype}")
    print(f"   X_ctx shape (Bins): {data['X_ctx'].shape}, dtype={data['X_ctx'].dtype}")

# 执行处理
if __name__ == "__main__":
    load_and_process_ctx()

🧬 Generating Baseline Context (X_ctx) for Dual-Head Model

📖 Loading input data: training_data_lincs2020_chemfiltered_1201_l.pkl...
   ✓ Samples: 509,006
   ✓ Genes: 978
   ✓ Target Plates: 4,583

📖 Reading Level 3 Control Data from: level3_beta_ctl_n188708x12328.gctx
   ✓ Format: Samples x Genes
   ✓ Loaded Control Matrix: (188708, 978)

📊 Computing Plate Baselines (Median)...
   ✓ Computed baselines for 4,576 plates

⚡ Generating Binned Context Embeddings...
   👉 Performing Rank-based Binning on Baselines...


   Mapping to samples: 100%|██████████| 509006/509006 [00:00<00:00, 627004.69it/s]


   ⚠️ Warning: 515 samples had missing plate baselines (filled with default).

💾 Saving to D:\科研\Models\drugreflector\processed_data\training_data_lincs2020_chemfiltered_1201_l_dual_head.pkl...
✅ Done! Output Dataset Keys: ['X', 'y', 'folds', 'sample_meta', 'metadata', 'gene_names', 'compound_names', 'pert_to_idx', 'X_ctx']
   X shape (Z-score): (509006, 978), dtype=float32
   X_ctx shape (Bins): (509006, 978), dtype=int32
